# `ALL`: Collect bigrams corresponding to top adverbs

In [1]:
import pandas as pd
from am_notebooks import (load_bigram_dfs, locate_bigram_am_paths,
                          nb_show_table, show_adv_bigrams)

from source.utils.associate import TOP_AM_DIR, adjust_ams
from source.utils.general import confirm_dir, print_iter, timestamp_today

ADV_FLOOR = 5000
K = 8

DATA_DATE = '2024-07-28'
TAG = 'ALL'
TOP_AM_TAG_DIR = TOP_AM_DIR / TAG
confirm_dir(TOP_AM_TAG_DIR)

data_top = f'{TAG}-Top{K}'
OUT_DIR = TOP_AM_TAG_DIR / data_top
confirm_dir(OUT_DIR)

# for loading `polar/*/bigram/*` tables
bigram_floor = 25 
mirror_floor = 5 
if TAG=='ALL':
    bigram_floor, mirror_floor = [x*2 for x in (bigram_floor, mirror_floor)]

## Set columns and diplay settings

In [2]:
FOCUS = ['f',
         'am_p1_given2', 'am_p1_given2_simple', 
         'conservative_log_ratio',
         'am_log_likelihood',
          'mutual_information',
         'am_odds_ratio_disc', 't_score',
         'N', 'f1', 'f2', 'E11', 'unexpected_f',
         'l1', 'l2', 
         'adv', 'adv_total', 'adj', 'adj_total']
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_columns', 9)
pd.set_option('display.width', 120)
pd.set_option("display.precision", 2)
pd.set_option("styler.format.precision", 2)
pd.set_option("styler.format.thousands", ",")
pd.set_option("display.float_format", '{:,.2f}'.format)

(MOVED the following to `./am_notebooks.py`)

In [3]:
# def force_ints(_df):
#     count_cols = _df.filter(regex=r'total|^[fN]').columns
#     _df[count_cols] = _df[count_cols].astype('int')
#     # _df[count_cols] = _df[:, count_cols].astype('int64')
#     # print(_df.dtypes.to_frame('dtypes'))
#     return _df


# def nb_show_table(df, n_dec: int = 2,
#                   adjust_columns: bool = True,
#                   outpath: Path = None,
#                   return_df: bool = False) -> None:
#     _df = df.copy()
#     try:
#         start_0 = _df.index.start == 0
#     except AttributeError:
#         pass
#     else:
#         _df.index.name = 'rank'
#         if start_0:
#             _df.index = _df.index + 1
#     if adjust_columns:
#         _df = adjust_assoc_columns(_df)
#     _df.columns = [f'`{c}`' for c in _df.columns]
#     _df.index = [f'**{r}**' for r in _df.index]
#     table = _df.to_markdown(floatfmt=f',.{n_dec}f', intfmt=',')
#     if outpath:
#         outpath.write_text(table)

#     print(f'\n{table}\n')
#     return (_df if return_df else None)
# from pprint import pprint
# from source.utils.dataframes import catify_hit_table as catify


# bigram_am_paths = {d.name:
              
#                   tuple(d.joinpath('bigram/extra').glob(
#                       f'*{TAG}*min{mirror_floor if d.name.endswith("mirror") else bigram_floor}x*extra.parq'))[0]
                  
#               for d in POLAR_DIR.iterdir()}

# pprint(bigram_am_paths)
# bigram_dfs = {n: catify(update_index(pd.read_parquet(p)), 
#                         reverse=True) 
#               for n, p in bigram_am_paths.items()}

# def show_adv_bigrams(sample_size, C, 
#                      bigram_dfs, 
#                      selector: str = 'dP1', 
#                      column_list: list = None) -> dict:
#     def _force_ints(_df):
#         count_cols = _df.filter(regex=r'total$|^[fN]').columns
#         _df.loc[:, count_cols] = _df.loc[:, count_cols].apply(
#             pd.to_numeric, downcast='unsigned')
#         return _df

#     def get_top_bigrams(bdf, adv, bigram_k):
#         bdf = _force_ints(bdf.loc[bdf.adv == adv, :])
#         top_by_metric = [bdf.nlargest(bigram_k * 2, m) for m in ['dP1', 'LRC']]
#         half_k = bigram_k // 2
#         adv_pat_bigrams = pd.concat(
#             [top_bigrams.head(half_k) for top_bigrams in top_by_metric]).drop_duplicates()
#         x = 0
#         while len(adv_pat_bigrams) < min(bigram_k, len(bdf)):
#             x += 1
#             next_ix = half_k + x
            
#             try:
#                 next_entries = [top_by_metric[0].iloc[[next_ix], :], 
#                             top_by_metric[1].iloc[[next_ix], :]]
#             except IndexError:
#                 print(f'All bigrams for {adv} retrieved.')
#                 break
#             else:
#                 adv_pat_bigrams = pd.concat((adv_pat_bigrams, 
#                                          *next_entries)).drop_duplicates()
#         return adv_pat_bigrams.head(bigram_k)

#     bigram_k = max(sample_size + 2, 10)
#     print(
#         f'## Top {bigram_k} "most negative" bigrams corresponding to top {sample_size} adverbs\n')
#     print(timestamp_today())
#     patterns = list(bigram_dfs.keys())
#     top_adverbs = C.index
#     bigram_samples = {adv: dict.fromkeys(
#         patterns + ['both', 'adj']) for adv in top_adverbs}
#     bigrams, adj = [], []

#     for rank, adv in enumerate(top_adverbs, start=1):
#         print(f'\n### {rank}. _{adv}_\n')
#         adj_for_adv = []
#         adv_top = None

#         for pat, bdf in bigram_dfs.items():
#             bdf = adjust_assoc_columns(
#                 bdf[[c for c in FOCUS + ['adj', 'adj_total', 'adv', 'adv_total'] if c in bdf.columns]])
#             bdf = bdf.loc[bdf.LRC >= 1, :]
#             adv_pat_bigrams = get_top_bigrams(bdf, adv, bigram_k)

#             if adv_pat_bigrams.empty:
#                 print(f'No bigrams found in loaded `{pat}` AM table.')
#             else:
#                 print(
#                     f'\n#### Top {len(adv_pat_bigrams)} `{pat}` "{adv}_*" bigrams (sorted by `{selector}`; `LRC > 1`)\n')
#                 column_list = column_list if column_list is not None else bdf.columns
#                 nb_show_table(adv_pat_bigrams.filter(column_list), n_dec=2)

#             adj_for_adv.extend(adv_pat_bigrams.adj.drop_duplicates().to_list())
#             bigram_samples[adv][pat] = adv_pat_bigrams
#             if adv_top is None: 
#                 adv_top = adv_pat_bigrams 
#             else:
#                 adv_top = pd.concat([df.fillna('') for df in (adv_top, adv_pat_bigrams)])

#         bigram_samples[adv]['adj'] = set(adj_for_adv)
#         bigrams.extend(adv_top.l2.drop_duplicates().to_list())
#         adj.extend(adj_for_adv)
#         bigram_samples[adv]['both'] = adv_top

#     bigram_samples['bigrams'] = set(bigrams)
#     bigram_samples['adj'] = set(adj)
#     return bigram_samples, bigram_k

## Load data

In [4]:
bigram_dfs = load_bigram_dfs(locate_bigram_am_paths(TAG, mirror_floor, bigram_floor))

try:
    combined_am_csv = tuple(OUT_DIR.glob(
                   f'{TAG}-Top{K}_NEG-ADV_combined-{ADV_FLOOR}*.{DATA_DATE or timestamp_today()}.csv'))[0]
except IndexError:
    combined_am_csv = tuple(TOP_AM_TAG_DIR.rglob(
        f'{TAG}-Top{K}_NEG-ADV_combined*.csv'))[0]
print(f'Loaded from: "{combined_am_csv}"')

C = adjust_ams(pd.read_csv(combined_am_csv, index_col='adv'))

main_cols_ordered = pd.concat((*[C.filter(like=m).columns.to_series() for m in ('LRC', 'P1', 'G2')],
                               *[C.filter(regex=f'^{f}_').columns.to_series() for f in ['f', 'f1', 'f2']])
                              ).to_list()
C

{'RBdirect': PosixPath('/share/compling/projects/sanpi/results/assoc_df/polar/RBdirect/bigram/extra/polarized-bigram_ALL-direct_min50x_extra.parq'),
 'mirror': PosixPath('/share/compling/projects/sanpi/results/assoc_df/polar/mirror/bigram/extra/polarized-bigram_ALL-mirror_min10x_extra.parq')}
Loaded from: "/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8_NEG-ADV_combined-5000.2024-07-28.csv"


,key_SET,f_SET,dP1_SET,P1_SET,...,r_f_MIR,r_N_MIR,r_f1_MIR,r_f2_MIR
adv,,,,,,,,,
necessarily,NEGany~necessarily,42595,0.83,0.87,...,0.02,0.02,0.09,0.02
that,NEGany~that,164768,0.75,0.79,...,0.03,0.02,0.09,0.03
exactly,NEGany~exactly,43813,0.70,0.75,...,0.02,0.02,0.09,0.02
any,NEGany~any,15384,0.40,0.45,...,0.07,0.02,0.09,0.03
remotely,NEGany~remotely,5661,0.30,0.34,...,0.33,0.02,0.09,0.14
ever,NEGany~ever,5932,0.01,0.05,...,0.79,0.02,0.09,0.04
yet,NEGany~yet,51867,0.50,0.54,...,0.01,0.02,0.09,0.01
immediately,NEGany~immediately,56099,0.54,0.58,...,0.01,0.02,0.09,0.01
particularly,NEGany~particularly,55527,0.07,0.11,...,0.17,0.02,0.09,0.03


In [5]:
samples_dict, bigram_k = show_adv_bigrams(
    K, C, bigram_dfs,
    column_list=[
        *pd.Series(main_cols_ordered).str.replace(
            r'mean_|_SET|_MIR', '', regex=True)
        .drop_duplicates().to_list(),
        'adv_total', 'adj_total',
        # 't', 'MI'
    ]
)

## Top 10 "most negative" bigrams corresponding to top 8 adverbs

2024-07-29

### 1. _necessarily_


#### Top 10 `RBdirect` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                       |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:--------------------------------------|--------:|--------:|-------:|---------:|------:|----------:|-------:|--------------:|--------------:|
| **NEGany~necessarily_useful**         |    6.75 |    0.96 |   1.00 |   651.75 |   104 | 3,173,660 |    104 |        48,947 |       227,709 |
| **NEGany~necessarily_fun**            |    6.62 |    0.96 |   1.00 |   601.61 |    96 | 3,173,660 |     96 |        48,947 |       190,026 |
| **NEGany~necessarily_essential**      |    6.57 |    0.96 |   1.00 |   582.81 |    93 | 3,173,660 |     93 |        48,947 |        69,845 |
| **NEGany~necessarily_reliable**       |    6.35 |    0.96 |   1.00 |   507.61 |    81 | 3,173,660 |     

## Top 10 "most negative" bigrams corresponding to top 8 adverbs

2024-07-28

### 1. _necessarily_


#### Top 10 `RBdirect` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                       |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:--------------------------------------|--------:|--------:|-------:|---------:|------:|----------:|-------:|--------------:|--------------:|
| **NEGany~necessarily_useful**         |    6.75 |    0.96 |   1.00 |   651.75 |   104 | 3,173,660 |    104 |        48,947 |       227,709 |
| **NEGany~necessarily_fun**            |    6.62 |    0.96 |   1.00 |   601.61 |    96 | 3,173,660 |     96 |        48,947 |       190,026 |
| **NEGany~necessarily_essential**      |    6.57 |    0.96 |   1.00 |   582.81 |    93 | 3,173,660 |     93 |        48,947 |        69,845 |
| **NEGany~necessarily_reliable**       |    6.35 |    0.96 |   1.00 |   507.61 |    81 | 3,173,660 |     81 |        48,947 |        90,598 |
| **NEGany~necessarily_proud**          |    6.14 |    0.96 |   1.00 |   444.94 |    71 | 3,173,660 |     71 |        48,947 |       207,536 |
| **NEGany~necessarily_indicative**     |    9.43 |    0.95 |   0.99 | 8,577.54 | 1,389 | 3,173,660 |  1,400 |        48,947 |         8,148 |
| **NEGany~necessarily_easy**           |    9.01 |    0.95 |   0.99 | 5,605.64 |   909 | 3,173,660 |    917 |        48,947 |       579,827 |
| **NEGany~necessarily_representative** |    8.34 |    0.95 |   0.99 | 2,996.58 |   487 | 3,173,660 |    492 |        48,947 |        18,355 |
| **NEGany~necessarily_surprising**     |    8.33 |    0.95 |   1.00 | 2,117.16 |   340 | 3,173,660 |    341 |        48,947 |        70,540 |
| **NEGany~necessarily_new**            |    7.94 |    0.94 |   0.98 | 2,923.82 |   482 | 3,173,660 |    492 |        48,947 |       253,862 |


#### Top 7 `mirror` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:-------------------------------|--------:|--------:|-------:|-------:|------:|--------:|-------:|--------------:|--------------:|
| **NEGmir~necessarily_right**   |    2.15 |    0.83 |   1.00 |  81.13 |    23 | 291,732 |     23 |         1,107 |         5,576 |
| **NEGmir~necessarily_illegal** |    1.20 |    0.83 |   1.00 |  52.91 |    15 | 291,732 |     15 |         1,107 |           937 |
| **NEGmir~necessarily_wrong**   |    5.04 |    0.81 |   0.98 | 698.74 |   211 | 291,732 |    216 |         1,107 |        20,880 |
| **NEGmir~necessarily_new**     |    1.84 |    0.79 |   0.96 |  73.19 |    23 | 291,732 |     24 |         1,107 |        12,836 |
| **NEGmir~necessarily_bad**     |    2.95 |    0.77 |   0.94 | 154.45 |    50 | 291,732 |     53 |         1,107 |        10,261 |
| **NEGmir~necessarily_true**    |    2.69 |    0.73 |   0.90 | 150.42 |    53 | 291,732 |     59 |         1,107 |         6,191 |
| **NEGmir~necessarily_better**  |    1.63 |    0.70 |   0.87 |  72.90 |    27 | 291,732 |     31 |         1,107 |        14,013 |


### 2. _that_


#### Top 10 `RBdirect` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                             |   `LRC` |   `dP1` |   `P1` |      `G2` |   `f` |      `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:----------------------------|--------:|--------:|-------:|----------:|------:|----------:|-------:|--------------:|--------------:|
| **NEGany~that_far-fetched** |    5.83 |    0.96 |   1.00 |    369.74 |    59 | 3,173,660 |     59 |       208,262 |         5,185 |
| **NEGany~that_thrilled**    |    5.83 |    0.96 |   1.00 |    369.74 |    59 | 3,173,660 |     59 |       208,262 |        24,182 |
| **NEGany~that_uncommon**    |    9.43 |    0.95 |   1.00 |  4,998.32 |   802 | 3,173,660 |    804 |       208,262 |        11,312 |
| **NEGany~that_surprising**  |    9.20 |    0.95 |   0.99 |  6,986.81 | 1,133 | 3,173,660 |  1,143 |       208,262 |        70,540 |
| **NEGany~that_dissimilar**  |    7.86 |    0.95 |   0.99 |  1,871.65 |   304 | 3,173,660 |    307 |       208,262 |         4,605 |
| **NEGany~that_unusual**     |    8.92 |    0.95 |   0.99 |  6,003.05 |   977 | 3,173,660 |    988 |       208,262 |        71,234 |
| **NEGany~that_complicated** |    8.68 |    0.94 |   0.98 |  7,337.84 | 1,207 | 3,173,660 |  1,230 |       208,262 |       159,822 |
| **NEGany~that_hard**        |    8.59 |    0.91 |   0.96 | 58,817.24 | 9,948 | 3,173,660 | 10,380 |       208,262 |       348,463 |
| **NEGany~that_familiar**    |    8.35 |    0.93 |   0.97 |  6,781.11 | 1,126 | 3,173,660 |  1,156 |       208,262 |       156,296 |
| **NEGany~that_noticeable**  |    7.65 |    0.95 |   0.99 |  1,621.81 |   264 | 3,173,660 |    267 |       208,262 |        31,467 |


#### Top 10 `mirror` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                             |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |    `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:----------------------------|--------:|--------:|-------:|---------:|------:|--------:|-------:|--------------:|--------------:|
| **NEGmir~that_keen**        |    2.73 |    0.83 |   1.00 |   109.35 |    31 | 291,732 |     31 |         5,494 |         1,360 |
| **NEGmir~that_impressive**  |    2.15 |    0.83 |   1.00 |    81.13 |    23 | 291,732 |     23 |         5,494 |         5,007 |
| **NEGmir~that_fond**        |    1.84 |    0.79 |   0.96 |    73.19 |    23 | 291,732 |     24 |         5,494 |         1,115 |
| **NEGmir~that_comfortable** |    1.84 |    0.79 |   0.96 |    73.19 |    23 | 291,732 |     24 |         5,494 |         4,642 |
| **NEGmir~that_clear**       |    1.30 |    0.78 |   0.95 |    56.03 |    18 | 291,732 |     19 |         5,494 |         6,722 |
| **NEGmir~that_simple**      |    4.36 |    0.72 |   0.90 | 1,340.19 |   474 | 291,732 |    529 |         5,494 |        25,408 |
| **NEGmir~that_easy**        |    4.23 |    0.71 |   0.89 | 1,248.84 |   450 | 291,732 |    508 |         5,494 |        18,610 |
| **NEGmir~that_great**       |    3.57 |    0.66 |   0.84 |   725.16 |   286 | 291,732 |    342 |         5,494 |         5,568 |
| **NEGmir~that_big**         |    3.17 |    0.69 |   0.86 |   300.54 |   113 | 291,732 |    131 |         5,494 |         8,177 |
| **NEGmir~that_popular**     |    3.15 |    0.76 |   0.93 |   195.15 |    65 | 291,732 |     70 |         5,494 |         5,668 |


### 3. _exactly_


#### Top 10 `RBdirect` "exactly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                               |   `LRC` |   `dP1` |   `P1` |      `G2` |   `f` |      `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:------------------------------|--------:|--------:|-------:|----------:|------:|----------:|-------:|--------------:|--------------:|
| **NEGany~exactly_conducive**  |    7.82 |    0.96 |   1.00 |  1,303.50 |   208 | 3,173,660 |    208 |        58,643 |         9,110 |
| **NEGany~exactly_shocking**   |    7.33 |    0.96 |   1.00 |    946.29 |   151 | 3,173,660 |    151 |        58,643 |        35,115 |
| **NEGany~exactly_pleasant**   |    7.24 |    0.96 |   1.00 |    889.88 |   142 | 3,173,660 |    142 |        58,643 |        52,223 |
| **NEGany~exactly_famous**     |    7.10 |    0.96 |   1.00 |    814.68 |   130 | 3,173,660 |    130 |        58,643 |       223,813 |
| **NEGany~exactly_difficult**  |    7.05 |    0.96 |   1.00 |    789.62 |   126 | 3,173,660 |    126 |        58,643 |       732,106 |
| **NEGany~exactly_easy**       |    9.32 |    0.95 |   0.99 |  6,596.91 | 1,066 | 3,173,660 |  1,073 |        58,643 |       579,827 |
| **NEGany~exactly_new**        |    9.10 |    0.94 |   0.99 |  8,410.32 | 1,371 | 3,173,660 |  1,388 |        58,643 |       253,862 |
| **NEGany~exactly_cheap**      |    8.96 |    0.95 |   0.99 |  4,281.59 |   691 | 3,173,660 |    695 |        58,643 |        60,531 |
| **NEGany~exactly_clear**      |    8.78 |    0.94 |   0.98 | 10,578.33 | 1,746 | 3,173,660 |  1,784 |        58,643 |       349,214 |
| **NEGany~exactly_surprising** |    8.71 |    0.95 |   1.00 |  2,743.34 |   440 | 3,173,660 |    441 |        58,643 |        70,540 |


#### Top 4 `mirror` "exactly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                          |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:-------------------------|--------:|--------:|-------:|-------:|------:|--------:|-------:|--------------:|--------------:|
| **NEGmir~exactly_sure**  |    5.31 |    0.83 |   1.00 | 522.11 |   148 | 291,732 |    148 |         1,041 |         6,761 |
| **NEGmir~exactly_easy**  |    1.86 |    0.83 |   1.00 |  70.55 |    20 | 291,732 |     20 |         1,041 |        18,610 |
| **NEGmir~exactly_clear** |    3.38 |    0.81 |   0.98 | 173.89 |    52 | 291,732 |     53 |         1,041 |         6,722 |
| **NEGmir~exactly_new**   |    2.31 |    0.80 |   0.97 |  93.90 |    29 | 291,732 |     30 |         1,041 |        12,836 |


### 4. _any_


#### Top 10 `RBdirect` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                         |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:------------------------|--------:|--------:|-------:|---------:|------:|----------:|-------:|--------------:|--------------:|
| **NEGany~any_happier**  |    6.34 |    0.82 |   0.86 | 4,420.49 |   828 | 3,173,660 |    963 |        34,382 |        16,606 |
| **NEGany~any_younger**  |    5.57 |    0.79 |   0.83 | 1,323.36 |   255 | 3,173,660 |    307 |        34,382 |        26,216 |
| **NEGany~any_nicer**    |    4.75 |    0.76 |   0.81 |   486.82 |    96 | 3,173,660 |    119 |        34,382 |         9,955 |
| **NEGany~any_simpler**  |    5.00 |    0.71 |   0.75 | 1,087.71 |   226 | 3,173,660 |    300 |        34,382 |        23,480 |
| **NEGany~any_brighter** |    3.91 |    0.67 |   0.72 |   292.00 |    63 | 3,173,660 |     88 |        34,382 |         9,280 |
| **NEGany~any_easier**   |    5.08 |    0.62 |   0.66 | 6,987.78 | 1,594 | 3,173,660 |  2,405 |        34,382 |       209,940 |
| **NEGany~any_cheaper**  |    4.01 |    0.58 |   0.63 |   542.97 |   129 | 3,173,660 |    206 |        34,382 |        46,055 |
| **NEGany~any_smarter**  |    4.00 |    0.63 |   0.67 |   394.95 |    89 | 3,173,660 |    132 |        34,382 |         8,501 |
| **NEGany~any_clearer**  |    4.26 |    0.54 |   0.58 | 1,421.60 |   355 | 3,173,660 |    608 |        34,382 |        11,680 |
| **NEGany~any_worse**    |    3.94 |    0.42 |   0.46 | 5,676.37 | 1,686 | 3,173,660 |  3,673 |        34,382 |       179,012 |


#### Top 10 `mirror` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                          |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |    `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:-------------------------|--------:|--------:|-------:|---------:|------:|--------:|-------:|--------------:|--------------:|
| **NEGmir~any_younger**   |    1.86 |    0.83 |   1.00 |    70.55 |    20 | 291,732 |     20 |         1,197 |           939 |
| **NEGmir~any_clearer**   |    1.50 |    0.83 |   1.00 |    59.97 |    17 | 291,732 |     17 |         1,197 |           130 |
| **NEGmir~any_different** |    3.24 |    0.81 |   0.98 |   159.93 |    48 | 291,732 |     49 |         1,197 |        36,166 |
| **NEGmir~any_bigger**    |    2.73 |    0.80 |   0.97 |   118.17 |    36 | 291,732 |     37 |         1,197 |         3,923 |
| **NEGmir~any_easier**    |    3.04 |    0.75 |   0.92 |   181.65 |    61 | 291,732 |     66 |         1,197 |         2,386 |
| **NEGmir~any_better**    |    4.38 |    0.74 |   0.91 | 1,096.01 |   380 | 291,732 |    419 |         1,197 |        14,013 |
| **NEGmir~any_worse**     |    2.73 |    0.66 |   0.83 |   217.46 |    87 | 291,732 |    105 |         1,197 |         8,790 |
| **NEGmir~any_higher**    |    1.42 |    0.74 |   0.91 |    61.24 |    21 | 291,732 |     23 |         1,197 |         2,893 |
| **NEGmir~any_closer**    |    2.33 |    0.65 |   0.83 |   141.82 |    57 | 291,732 |     69 |         1,197 |           993 |
| **NEGmir~any_good**      |    2.12 |    0.74 |   0.91 |    93.53 |    32 | 291,732 |     35 |         1,197 |        31,585 |


### 5. _remotely_


#### Top 10 `RBdirect` "remotely_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:-------------------------------|--------:|--------:|-------:|---------:|------:|----------:|-------:|--------------:|--------------:|
| **NEGany~remotely_surprising** |    5.07 |    0.85 |   0.89 |   413.61 |    75 | 3,173,660 |     84 |        16,426 |        70,540 |
| **NEGany~remotely_ready**      |    4.16 |    0.74 |   0.78 |   287.63 |    58 | 3,173,660 |     74 |        16,426 |       141,590 |
| **NEGany~remotely_true**       |    4.61 |    0.63 |   0.68 | 1,111.13 |   250 | 3,173,660 |    370 |        16,426 |       231,639 |
| **NEGany~remotely_funny**      |    4.15 |    0.60 |   0.65 |   589.74 |   137 | 3,173,660 |    212 |        16,426 |       122,927 |
| **NEGany~remotely_qualified**  |    3.13 |    0.52 |   0.56 |   222.79 |    57 | 3,173,660 |    101 |        16,426 |        74,643 |
| **NEGany~remotely_close**      |    3.65 |    0.39 |   0.43 | 2,243.22 |   694 | 3,173,660 |  1,597 |        16,426 |       411,329 |
| **NEGany~remotely_comparable** |    2.98 |    0.38 |   0.43 |   375.73 |   118 | 3,173,660 |    277 |        16,426 |        12,252 |
| **NEGany~remotely_accurate**   |    2.10 |    0.33 |   0.37 |   143.78 |    50 | 3,173,660 |    134 |        16,426 |       152,299 |
| **NEGany~remotely_interested** |    2.74 |    0.27 |   0.31 |   817.06 |   330 | 3,173,660 |  1,062 |        16,426 |       264,528 |
| **NEGany~remotely_similar**    |    2.20 |    0.23 |   0.27 |   334.61 |   152 | 3,173,660 |    559 |        16,426 |       203,453 |


#### Top 10 `mirror` "remotely_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                 |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:--------------------------------|--------:|--------:|-------:|-------:|------:|--------:|-------:|--------------:|--------------:|
| **NEGmir~remotely_believable**  |    1.96 |    0.83 |   1.00 |  74.08 |    21 | 291,732 |     21 |         2,341 |           600 |
| **NEGmir~remotely_surprising**  |    1.50 |    0.83 |   1.00 |  59.97 |    17 | 291,732 |     17 |         2,341 |         2,662 |
| **NEGmir~remotely_comparable**  |    2.72 |    0.76 |   0.94 | 134.02 |    44 | 291,732 |     47 |         2,341 |           283 |
| **NEGmir~remotely_true**        |    3.04 |    0.75 |   0.92 | 181.65 |    61 | 291,732 |     66 |         2,341 |         6,191 |
| **NEGmir~remotely_new**         |    1.00 |    0.69 |   0.86 |  50.62 |    19 | 291,732 |     22 |         2,341 |        12,836 |
| **NEGmir~remotely_close**       |    3.28 |    0.65 |   0.82 | 532.96 |   218 | 291,732 |    267 |         2,341 |        13,874 |
| **NEGmir~remotely_funny**       |    1.85 |    0.63 |   0.80 |  97.91 |    41 | 291,732 |     51 |         2,341 |         5,365 |
| **NEGmir~remotely_similar**     |    1.70 |    0.49 |   0.66 | 127.32 |    71 | 291,732 |    107 |         2,341 |         7,011 |
| **NEGmir~remotely_possible**    |    1.42 |    0.56 |   0.73 |  78.73 |    38 | 291,732 |     52 |         2,341 |         3,160 |
| **NEGmir~remotely_interesting** |    1.80 |    0.56 |   0.73 | 115.20 |    56 | 291,732 |     77 |         2,341 |        12,447 |


### 6. _ever_


#### Top 10 `RBdirect` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                           |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |       `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:--------------------------|--------:|--------:|-------:|---------:|------:|-----------:|-------:|--------------:|--------------:|
| **NEGany~ever_simple**    |    5.56 |    0.80 |   0.84 | 1,109.28 |   211 |  3,173,660 |    250 |       114,075 |       396,749 |
| **NEGany~ever_boring**    |    4.46 |    0.76 |   0.80 |   362.74 |    72 |  3,173,660 |     90 |       114,075 |        45,891 |
| **NEGany~ever_easy**      |    5.41 |    0.73 |   0.77 | 2,105.13 |   429 |  3,173,660 |    555 |       114,075 |       579,827 |
| **NEGany~ever_certain**   |    4.80 |    0.72 |   0.76 |   713.34 |   147 |  3,173,660 |    193 |       114,075 |        74,952 |
| **NEGany~ever_sure**      |    3.34 |    0.49 |   0.54 |   328.20 |    87 |  3,173,660 |    162 |       114,075 |       262,825 |
| **NEGany~ever_good**      |    3.81 |    0.46 |   0.50 | 1,188.69 |   331 |  3,173,660 |    660 |       114,075 |     1,681,795 |
| **COM~ever_closer**       |    3.51 |    0.04 |   1.00 |   538.66 | 6,305 | 69,662,736 |  6,307 |       114,075 |        61,475 |
| **NEGany~ever_enough**    |    3.54 |    0.45 |   0.50 |   618.62 |   173 |  3,173,660 |    347 |       114,075 |       152,020 |
| **NEGany~ever_perfect**   |    3.34 |    0.40 |   0.44 |   706.71 |   216 |  3,173,660 |    489 |       114,075 |       104,659 |
| **NEGany~ever_satisfied** |    2.57 |    0.38 |   0.42 |   203.01 |    64 |  3,173,660 |    151 |       114,075 |        62,862 |


#### Top 10 `mirror` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                         |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |    `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:------------------------|--------:|--------:|-------:|---------:|------:|--------:|-------:|--------------:|--------------:|
| **NEGmir~ever_simple**  |    5.82 |    0.83 |   1.00 |   726.76 |   206 | 291,732 |    206 |         5,060 |        25,408 |
| **NEGmir~ever_enough**  |    5.30 |    0.83 |   1.00 |   518.58 |   147 | 291,732 |    147 |         5,060 |         2,596 |
| **NEGmir~ever_certain** |    5.26 |    0.83 |   1.00 |   504.47 |   143 | 291,732 |    143 |         5,060 |         1,800 |
| **NEGmir~ever_boring**  |    3.80 |    0.83 |   1.00 |   201.07 |    57 | 291,732 |     57 |         5,060 |         1,961 |
| **NEGmir~ever_black**   |    3.77 |    0.83 |   1.00 |   197.54 |    56 | 291,732 |     56 |         5,060 |         1,412 |
| **NEGmir~ever_easy**    |    6.44 |    0.83 |   1.00 | 1,285.01 |   368 | 291,732 |    369 |         5,060 |        18,610 |
| **NEGmir~ever_good**    |    5.68 |    0.82 |   0.99 | 1,013.87 |   299 | 291,732 |    303 |         5,060 |        31,585 |
| **NEGmir~ever_perfect** |    5.57 |    0.82 |   1.00 |   714.47 |   206 | 291,732 |    207 |         5,060 |         3,134 |
| **NEGmir~ever_sick**    |    3.15 |    0.83 |   1.00 |   137.57 |    39 | 291,732 |     39 |         5,060 |         1,895 |
| **NEGmir~ever_able**    |    4.17 |    0.78 |   0.95 |   426.52 |   136 | 291,732 |    143 |         5,060 |         3,704 |


### 7. _yet_


#### Top 10 `RBdirect` "yet_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                           |   `LRC` |   `dP1` |   `P1` |      `G2` |    `f` |      `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:--------------------------|--------:|--------:|-------:|----------:|-------:|----------:|-------:|--------------:|--------------:|
| **NEGany~yet_eligible**   |    8.96 |    0.96 |   1.00 |  2,807.56 |    448 | 3,173,660 |    448 |        95,763 |        23,252 |
| **NEGany~yet_official**   |    8.61 |    0.96 |   1.00 |  2,205.93 |    352 | 3,173,660 |    352 |        95,763 |         6,853 |
| **NEGany~yet_convinced**  |    7.50 |    0.96 |   1.00 |  1,059.09 |    169 | 3,173,660 |    169 |        95,763 |        12,132 |
| **NEGany~yet_online**     |    6.66 |    0.96 |   1.00 |    614.14 |     98 | 3,173,660 |     98 |        95,763 |        15,650 |
| **NEGany~yet_mainstream** |    6.11 |    0.96 |   1.00 |    438.67 |     70 | 3,173,660 |     70 |        95,763 |        17,792 |
| **NEGany~yet_clear**      |   10.77 |    0.95 |   0.99 | 64,409.97 | 10,406 | 3,173,660 | 10,476 |        95,763 |       349,214 |
| **NEGany~yet_ready**      |    9.93 |    0.94 |   0.99 | 45,985.07 |  7,501 | 3,173,660 |  7,599 |        95,763 |       141,590 |
| **NEGany~yet_complete**   |    9.20 |    0.94 |   0.98 | 13,277.09 |  2,174 | 3,173,660 |  2,208 |        95,763 |        86,361 |
| **NEGany~yet_final**      |    8.97 |    0.95 |   1.00 |  3,972.92 |    640 | 3,173,660 |    643 |        95,763 |         5,860 |
| **NEGany~yet_over**       |    7.21 |    0.95 |   0.99 |  1,003.13 |    162 | 3,173,660 |    163 |        95,763 |         3,983 |


#### Top 5 `mirror` "yet_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                          |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:-------------------------|--------:|--------:|-------:|-------:|------:|--------:|-------:|--------------:|--------------:|
| **NEGmir~yet_available** |    2.54 |    0.83 |   1.00 |  98.77 |    28 | 291,732 |     28 |           815 |        10,284 |
| **NEGmir~yet_certain**   |    1.96 |    0.83 |   1.00 |  74.08 |    21 | 291,732 |     21 |           815 |         1,800 |
| **NEGmir~yet_sure**      |    1.75 |    0.83 |   1.00 |  67.02 |    19 | 291,732 |     19 |           815 |         6,761 |
| **NEGmir~yet_clear**     |    1.75 |    0.83 |   1.00 |  67.02 |    19 | 291,732 |     19 |           815 |         6,722 |
| **NEGmir~yet_ready**     |    1.63 |    0.83 |   1.00 |  63.49 |    18 | 291,732 |     18 |           815 |         3,034 |


### 8. _immediately_


#### Top 10 `RBdirect` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                    |   `LRC` |   `dP1` |   `P1` |       `G2` |    `f` |      `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:-----------------------------------|--------:|--------:|-------:|-----------:|-------:|----------:|-------:|--------------:|--------------:|
| **NEGany~immediately_possible**    |    7.62 |    0.91 |   0.95 |   5,845.77 |  1,000 | 3,173,660 |  1,054 |        96,973 |       245,272 |
| **NEGany~immediately_clear**       |    8.16 |    0.89 |   0.94 | 141,186.69 | 24,416 | 3,173,660 | 26,038 |        96,973 |       349,214 |
| **NEGany~immediately_reachable**   |    5.56 |    0.86 |   0.91 |     610.53 |    109 | 3,173,660 |    120 |        96,973 |         2,672 |
| **NEGany~immediately_sure**        |    5.40 |    0.82 |   0.86 |     738.65 |    138 | 3,173,660 |    160 |        96,973 |       262,825 |
| **NEGany~immediately_certain**     |    4.19 |    0.71 |   0.75 |     336.68 |     70 | 3,173,660 |     93 |        96,973 |        74,952 |
| **NEGany~immediately_available**   |    5.70 |    0.67 |   0.72 |  98,046.67 | 21,078 | 3,173,660 | 29,351 |        96,973 |       666,909 |
| **NEGany~immediately_able**        |    4.70 |    0.59 |   0.63 |   2,655.18 |    626 | 3,173,660 |    989 |        96,973 |       223,196 |
| **NEGany~immediately_obvious**     |    4.55 |    0.52 |   0.56 |   8,712.07 |  2,238 | 3,173,660 |  3,989 |        96,973 |       165,439 |
| **NEGany~immediately_forthcoming** |    3.89 |    0.55 |   0.60 |     540.70 |    133 | 3,173,660 |    223 |        96,973 |         7,473 |
| **NEGany~immediately_intuitive**   |    3.21 |    0.55 |   0.59 |     218.74 |     54 | 3,173,660 |     91 |        96,973 |        20,664 |


#### Top 2 `mirror` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                  |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:---------------------------------|--------:|--------:|-------:|-------:|------:|--------:|-------:|--------------:|--------------:|
| **NEGmir~immediately_clear**     |    1.13 |    0.55 |   0.72 |  62.95 |    31 | 291,732 |     43 |         1,195 |         6,722 |
| **NEGmir~immediately_available** |    1.85 |    0.42 |   0.59 | 241.53 |   162 | 291,732 |    275 |         1,195 |        10,284 |


### 9. _particularly_


#### Top 10 `RBdirect` "particularly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                       |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:--------------------------------------|--------:|--------:|-------:|---------:|------:|----------:|-------:|--------------:|--------------:|
| **NEGany~particularly_likable**       |    5.53 |    0.85 |   0.89 |   657.49 |   119 | 3,173,660 |    133 |       513,668 |         8,160 |
| **NEGany~particularly_likeable**      |    5.34 |    0.84 |   0.88 |   579.07 |   106 | 3,173,660 |    120 |       513,668 |         5,902 |
| **NEGany~particularly_forthcoming**   |    4.85 |    0.82 |   0.87 |   387.25 |    72 | 3,173,660 |     83 |       513,668 |         7,473 |
| **NEGany~particularly_religious**     |    6.12 |    0.81 |   0.86 | 2,585.71 |   485 | 3,173,660 |    565 |       513,668 |        28,028 |
| **NEGany~particularly_original**      |    5.86 |    0.80 |   0.85 | 1,894.60 |   360 | 3,173,660 |    426 |       513,668 |        37,594 |
| **NEGany~particularly_new**           |    6.04 |    0.79 |   0.83 | 3,874.46 |   747 | 3,173,660 |    900 |       513,668 |       253,862 |
| **NEGany~particularly_athletic**      |    4.77 |    0.75 |   0.79 |   541.01 |   108 | 3,173,660 |    136 |       513,668 |        17,142 |
| **NEGany~particularly_revolutionary** |    4.49 |    0.75 |   0.79 |   385.60 |    77 | 3,173,660 |     97 |       513,668 |        10,338 |
| **NEGany~particularly_flashy**        |    4.08 |    0.73 |   0.77 |   278.96 |    57 | 3,173,660 |     74 |       513,668 |         4,494 |
| **NEGany~particularly_surprising**    |    4.71 |    0.57 |   0.61 | 4,433.52 | 1,069 | 3,173,660 |  1,743 |       513,668 |        70,540 |


#### Top 10 `mirror` "particularly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                       |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |    `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:--------------------------------------|--------:|--------:|-------:|---------:|------:|--------:|-------:|--------------:|--------------:|
| **NEGmir~particularly_novel**         |    3.71 |    0.83 |   1.00 |   190.49 |    54 | 291,732 |     54 |        13,003 |           320 |
| **NEGmir~particularly_comfortable**   |    3.36 |    0.83 |   1.00 |   155.21 |    44 | 291,732 |     44 |        13,003 |         4,642 |
| **NEGmir~particularly_revolutionary** |    3.15 |    0.83 |   1.00 |   137.57 |    39 | 291,732 |     39 |        13,003 |           485 |
| **NEGmir~particularly_radical**       |    2.73 |    0.83 |   1.00 |   109.35 |    31 | 291,732 |     31 |        13,003 |         1,072 |
| **NEGmir~particularly_fast**          |    2.39 |    0.83 |   1.00 |    91.71 |    26 | 291,732 |     26 |        13,003 |         1,259 |
| **NEGmir~particularly_new**           |    5.92 |    0.81 |   0.99 | 1,365.17 |   404 | 291,732 |    410 |        13,003 |        12,836 |
| **NEGmir~particularly_surprising**    |    5.06 |    0.82 |   0.99 |   564.67 |   166 | 291,732 |    168 |        13,003 |         2,662 |
| **NEGmir~particularly_wrong**         |    5.05 |    0.81 |   0.98 |   702.22 |   212 | 291,732 |    217 |        13,003 |        20,880 |
| **NEGmir~particularly_remarkable**    |    4.24 |    0.80 |   0.97 |   354.53 |   108 | 291,732 |    111 |        13,003 |         3,238 |
| **NEGmir~particularly_close**         |    4.01 |    0.77 |   0.94 |   415.70 |   136 | 291,732 |    145 |        13,003 |        13,874 |


### 10. _inherently_

All bigrams for inherently retrieved.

#### Top 7 `RBdirect` "inherently_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                   |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:----------------------------------|--------:|--------:|-------:|---------:|------:|----------:|-------:|--------------:|--------------:|
| **NEGany~inherently_wrong**       |    5.36 |    0.66 |   0.71 | 7,535.97 | 1,639 | 3,173,660 |  2,315 |        47,803 |       149,064 |
| **NEGany~inherently_illegal**     |    3.51 |    0.60 |   0.64 |   252.59 |    59 | 3,173,660 |     92 |        47,803 |        30,194 |
| **NEGany~inherently_bad**         |    4.62 |    0.56 |   0.61 | 3,270.68 |   794 | 3,173,660 |  1,307 |        47,803 |       429,537 |
| **NEGany~inherently_negative**    |    2.50 |    0.35 |   0.39 |   222.63 |    75 | 3,173,660 |    193 |        47,803 |        53,385 |
| **NEGany~inherently_evil**        |    2.81 |    0.28 |   0.32 |   905.80 |   358 | 3,173,660 |  1,123 |        47,803 |        22,706 |
| **NEGany~inherently_good**        |    2.21 |    0.20 |   0.24 |   554.72 |   283 | 3,173,660 |  1,177 |        47,803 |     1,681,795 |
| **NEGany~inherently_problematic** |    1.17 |    0.17 |   0.21 |    98.70 |    58 | 3,173,660 |    277 |        47,803 |        33,408 |

All bigrams for inherently retrieved.

#### Top 6 `mirror` "inherently_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |    `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:-------------------------------|--------:|--------:|-------:|---------:|------:|--------:|-------:|--------------:|--------------:|
| **NEGmir~inherently_improper** |    1.63 |    0.83 |   1.00 |    63.49 |    18 | 291,732 |     18 |         5,133 |           142 |
| **NEGmir~inherently_illegal**  |    2.10 |    0.79 |   0.96 |    83.54 |    26 | 291,732 |     27 |         5,133 |           937 |
| **NEGmir~inherently_wrong**    |    4.08 |    0.66 |   0.83 | 3,787.53 | 1,513 | 291,732 |  1,826 |         5,133 |        20,880 |
| **NEGmir~inherently_bad**      |    2.86 |    0.62 |   0.79 |   342.53 |   148 | 291,732 |    188 |         5,133 |        10,261 |
| **NEGmir~inherently_better**   |    1.65 |    0.57 |   0.75 |    93.95 |    44 | 291,732 |     59 |         5,133 |        14,013 |
| **NEGmir~inherently_evil**     |    1.18 |    0.41 |   0.59 |    85.70 |    58 | 291,732 |     99 |         5,133 |         1,271 |


### 11. _terribly_


#### Top 10 `RBdirect` "terribly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:-------------------------------|--------:|--------:|-------:|---------:|------:|----------:|-------:|--------------:|--------------:|
| **NEGany~terribly_surprising** |    8.66 |    0.94 |   0.98 | 5,794.10 |   949 | 3,173,660 |    964 |        58,964 |        70,540 |
| **NEGany~terribly_uncommon**   |    6.33 |    0.94 |   0.98 |   625.85 |   103 | 3,173,660 |    105 |        58,964 |        11,312 |
| **NEGany~terribly_likely**     |    6.26 |    0.93 |   0.97 |   649.50 |   108 | 3,173,660 |    111 |        58,964 |       890,421 |
| **NEGany~terribly_productive** |    5.39 |    0.91 |   0.96 |   376.84 |    64 | 3,173,660 |     67 |        58,964 |       102,361 |
| **NEGany~terribly_reliable**   |    4.99 |    0.90 |   0.94 |   296.70 |    51 | 3,173,660 |     54 |        58,964 |        90,598 |
| **NEGany~terribly_original**   |    6.41 |    0.90 |   0.94 | 1,150.48 |   199 | 3,173,660 |    212 |        58,964 |        37,594 |
| **NEGany~terribly_different**  |    6.33 |    0.85 |   0.89 | 2,022.47 |   366 | 3,173,660 |    409 |        58,964 |       825,838 |
| **NEGany~terribly_unusual**    |    6.17 |    0.90 |   0.94 |   847.05 |   146 | 3,173,660 |    155 |        58,964 |        71,234 |
| **NEGany~terribly_sure**       |    5.08 |    0.86 |   0.91 |   386.31 |    69 | 3,173,660 |     76 |        58,964 |       262,825 |
| **NEGany~terribly_impressive** |    5.97 |    0.86 |   0.90 | 1,087.64 |   196 | 3,173,660 |    218 |        58,964 |       178,051 |


#### Top 10 `mirror` "terribly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                 |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |   `adv_total` |   `adj_total` |
|:--------------------------------|--------:|--------:|-------:|-------:|------:|--------:|-------:|--------------:|--------------:|
| **NEGmir~terribly_surprising**  |    4.07 |    0.83 |   1.00 | 236.35 |    67 | 291,732 |     67 |         4,610 |         2,662 |
| **NEGmir~terribly_original**    |    3.40 |    0.83 |   1.00 | 158.74 |    45 | 291,732 |     45 |         4,610 |         1,555 |
| **NEGmir~terribly_special**     |    2.23 |    0.83 |   1.00 |  84.66 |    24 | 291,732 |     24 |         4,610 |        15,541 |
| **NEGmir~terribly_unusual**     |    2.23 |    0.83 |   1.00 |  84.66 |    24 | 291,732 |     24 |         4,610 |         2,302 |
| **NEGmir~terribly_popular**     |    1.75 |    0.83 |   1.00 |  67.02 |    19 | 291,732 |     19 |         4,610 |         5,668 |
| **NEGmir~terribly_new**         |    3.66 |    0.80 |   0.97 | 225.93 |    69 | 291,732 |     71 |         4,610 |        12,836 |
| **NEGmir~terribly_interesting** |    2.44 |    0.68 |   0.85 | 145.16 |    56 | 291,732 |     66 |         4,610 |        12,447 |
| **NEGmir~terribly_interested**  |    2.36 |    0.74 |   0.91 | 112.46 |    39 | 291,732 |     43 |         4,610 |         8,255 |
| **NEGmir~terribly_clear**       |    1.20 |    0.83 |   1.00 |  52.91 |    15 | 291,732 |     15 |         4,610 |         6,722 |
| **NEGmir~terribly_remarkable**  |    1.03 |    0.83 |   1.00 |  49.38 |    14 | 291,732 |     14 |         4,610 |         3,238 |



 ### `2024-05-23` Top 10 "most negative" bigrams corresponding to top 5 adverbs



#### 1. _necessarily_


Top 10 `RBdirect` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                                       | `adj`          |   `adj_total` |   `LRC` |   `dP1` |      `G2` |   `f` |      `f1` |   `f2` |
 |:--------------------------------------|:---------------|--------------:|--------:|--------:|----------:|------:|----------:|-------:|
 | **NEGany~necessarily_sure**           | sure           |    844,981.00 |    5.91 |    0.95 |  1,436.68 |   222 | 3,226,213 |    224 |
 | **NEGany~necessarily_surprising**     | surprising     |    150,067.00 |    7.22 |    0.93 |  2,150.86 |   343 | 3,226,213 |    355 |
 | **NEGany~necessarily_indicative**     | indicative     |     12,760.00 |    8.37 |    0.93 |  8,811.69 | 1,406 | 3,226,213 |  1,456 |
 | **NEGany~necessarily_representative** | representative |     25,187.00 |    7.31 |    0.91 |  3,044.27 |   496 | 3,226,213 |    524 |
 | **NEGany~necessarily_available**      | available      |    866,272.00 |    6.36 |    0.89 |  1,280.24 |   213 | 3,226,213 |    230 |
 | **NEGany~necessarily_easy**           | easy           |    771,307.00 |    7.26 |    0.88 |  5,448.34 |   914 | 3,226,213 |    996 |
 | **NEGany~necessarily_true**           | true           |    348,994.00 |    6.89 |    0.82 | 18,199.76 | 3,238 | 3,226,213 |  3,786 |
 | **NEGany~necessarily_illegal**        | illegal        |     44,028.00 |    6.48 |    0.87 |  1,659.90 |   280 | 3,226,213 |    307 |
 | **NEGany~necessarily_related**        | related        |    137,661.00 |    6.74 |    0.84 |  4,271.76 |   742 | 3,226,213 |    842 |
 | **NEGany~necessarily_interested**     | interested     |    364,497.00 |    6.77 |    0.87 |  2,500.26 |   422 | 3,226,213 |    463 |


Top 3 `NEGmirror` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                              | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `G2` |   `f` |    `f1` |   `f2` |
 |:-----------------------------|:--------|--------------:|--------:|--------:|-------:|------:|--------:|-------:|
 | **NEGmir~necessarily_wrong** | wrong   |     20,866.00 |    4.27 |    0.81 | 708.98 |   209 | 289,770 |    214 |
 | **NEGmir~necessarily_bad**   | bad     |     10,783.00 |    2.02 |    0.76 | 153.43 |    50 | 289,770 |     54 |
 | **NEGmir~necessarily_true**  | true    |      7,402.00 |    2.18 |    0.75 | 159.07 |    53 | 289,770 |     58 |


#### 2. _exactly_


Top 10 `RBdirect` "exactly_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                               | `adj`      |   `adj_total` |   `LRC` |   `dP1` |      `G2` |   `f` |      `f1` |   `f2` |
 |:------------------------------|:-----------|--------------:|--------:|--------:|----------:|------:|----------:|-------:|
 | **NEGany~exactly_surprising** | surprising |    150,067.00 |    7.34 |    0.96 |  2,863.35 |   441 | 3,226,213 |    444 |
 | **NEGany~exactly_cheap**      | cheap      |     83,765.00 |    8.28 |    0.95 |  4,443.27 |   693 | 3,226,213 |    704 |
 | **NEGany~exactly_subtle**     | subtle     |     56,845.00 |    6.92 |    0.94 |  1,671.02 |   264 | 3,226,213 |    271 |
 | **NEGany~exactly_fun**        | fun        |    224,457.00 |    6.67 |    0.94 |  1,423.92 |   225 | 3,226,213 |    231 |
 | **NEGany~exactly_conducive**  | conducive  |     16,405.00 |    6.56 |    0.93 |  1,313.09 |   208 | 3,226,213 |    214 |
 | **NEGany~exactly_sure**       | sure       |    844,981.00 |    8.63 |    0.92 | 54,750.58 | 8,860 | 3,226,213 |  9,301 |
 | **NEGany~exactly_new**        | new        |    321,311.00 |    8.54 |    0.93 |  8,697.93 | 1,378 | 3,226,213 |  1,418 |
 | **NEGany~exactly_easy**       | easy       |    771,307.00 |    8.37 |    0.93 |  6,747.64 | 1,069 | 3,226,213 |  1,100 |
 | **NEGany~exactly_clear**      | clear      |    491,108.00 |    8.30 |    0.92 | 10,937.16 | 1,759 | 3,226,213 |  1,835 |
 | **NEGany~exactly_happy**      | happy      |    528,511.00 |    7.16 |    0.90 |  2,694.69 |   441 | 3,226,213 |    468 |

 No bigrams found in loaded `NEGmirror` AM table.

#### 3. _that_


Top 10 `RBdirect` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                             | `adj`       |   `adj_total` |   `LRC` |   `dP1` |      `G2` |   `f` |      `f1` |   `f2` |
 |:----------------------------|:------------|--------------:|--------:|--------:|----------:|------:|----------:|-------:|
 | **NEGany~that_uncommon**    | uncommon    |     61,767.00 |    8.39 |    0.94 |  5,136.91 |   804 | 3,226,213 |    819 |
 | **NEGany~that_fond**        | fond        |     39,809.00 |    7.27 |    0.94 |  2,127.94 |   334 | 3,226,213 |    341 |
 | **NEGany~that_surprising**  | surprising  |    150,067.00 |    8.14 |    0.92 |  7,115.30 | 1,141 | 3,226,213 |  1,187 |
 | **NEGany~that_common**      | common      |    556,435.00 |    8.12 |    0.92 |  7,564.08 | 1,216 | 3,226,213 |  1,268 |
 | **NEGany~that_dissimilar**  | dissimilar  |      8,816.00 |    7.00 |    0.92 |  1,904.15 |   307 | 3,226,213 |    321 |
 | **NEGany~that_hard**        | hard        |    430,990.00 |    7.96 |    0.88 | 59,642.82 | 9,966 | 3,226,213 | 10,818 |
 | **NEGany~that_complicated** | complicated |    180,071.00 |    7.95 |    0.91 |  7,450.89 | 1,208 | 3,226,213 |  1,270 |
 | **NEGany~that_impressed**   | impressed   |    113,281.00 |    7.57 |    0.91 |  4,207.58 |   684 | 3,226,213 |    721 |
 | **NEGany~that_noticeable**  | noticeable  |     40,372.00 |    6.78 |    0.91 |  1,632.07 |   265 | 3,226,213 |    279 |
 | **NEGany~that_exciting**    | exciting    |    236,396.00 |    7.48 |    0.90 |  4,892.83 |   805 | 3,226,213 |    859 |


Top 10 `NEGmirror` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                            | `adj`      |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |    `f1` |   `f2` |
 |:---------------------------|:-----------|--------------:|--------:|--------:|---------:|------:|--------:|-------:|
 | **NEGmir~that_popular**    | popular    |      5,787.00 |    2.50 |    0.76 |   200.44 |    65 | 289,770 |     70 |
 | **NEGmir~that_interested** | interested |      9,258.00 |    2.42 |    0.76 |   190.06 |    62 | 289,770 |     67 |
 | **NEGmir~that_difficult**  | difficult  |     16,043.00 |    2.15 |    0.75 |   155.64 |    52 | 289,770 |     57 |
 | **NEGmir~that_hard**       | hard       |      7,311.00 |    2.31 |    0.74 |   168.31 |    57 | 289,770 |     63 |
 | **NEGmir~that_close**      | close      |     13,962.00 |    2.39 |    0.73 |   174.26 |    60 | 289,770 |     67 |
 | **NEGmir~that_simple**     | simple     |     25,382.00 |    4.34 |    0.73 | 1,370.94 |   473 | 289,770 |    529 |
 | **NEGmir~that_easy**       | easy       |     20,050.00 |    4.21 |    0.72 | 1,258.15 |   442 | 289,770 |    500 |
 | **NEGmir~that_great**      | great      |      5,819.00 |    3.52 |    0.67 |   728.46 |   282 | 289,770 |    340 |
 | **NEGmir~that_good**       | good       |     33,540.00 |    3.07 |    0.56 |   953.31 |   447 | 289,770 |    615 |
 | **NEGmir~that_big**        | big        |      7,859.00 |    3.06 |    0.70 |   309.58 |   113 | 289,770 |    131 |


#### 4. _before_

 No bigrams found in loaded `RBdirect` AM table.
 No bigrams found in loaded `NEGmirror` AM table.

#### 5. _any_


Top 10 `RBdirect` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |      `f1` |   `f2` |
 |:-------------------------|:----------|--------------:|--------:|--------:|---------:|------:|----------:|-------:|
 | **NEGany~any_happier**   | happier   |     19,501.00 |    4.65 |    0.53 | 3,488.76 |   830 | 3,226,213 |  1,472 |
 | **NEGany~any_simpler**   | simpler   |     26,094.00 |    3.09 |    0.30 |   671.74 |   228 | 3,226,213 |    672 |
 | **NEGany~any_clearer**   | clearer   |     13,369.00 |    3.21 |    0.30 | 1,051.22 |   357 | 3,226,213 |  1,053 |
 | **NEGany~any_different** | different |    909,864.00 |    2.98 |    0.24 | 2,270.24 |   910 | 3,226,213 |  3,313 |
 | **NEGany~any_younger**   | younger   |     29,805.00 |    2.37 |    0.19 |   544.17 |   256 | 3,226,213 |  1,121 |
 | **NEGany~any_worse**     | worse     |    214,166.00 |    2.47 |    0.16 | 3,165.88 | 1,693 | 3,226,213 |  8,487 |
 | **NEGany~any_bigger**    | bigger    |    130,470.00 |    2.27 |    0.17 |   688.06 |   357 | 3,226,213 |  1,735 |
 | **NEGany~any_harder**    | harder    |     99,332.00 |    1.98 |    0.15 |   395.22 |   227 | 3,226,213 |  1,221 |
 | **NEGany~any_safer**     | safer     |     26,779.00 |    1.73 |    0.12 |   346.68 |   235 | 3,226,213 |  1,471 |
 | **NEGany~any_easier**    | easier    |    237,680.00 |    1.95 |    0.11 | 2,164.75 | 1,607 | 3,226,213 | 10,860 |


Top 4 `NEGmirror` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                       | `adj`   |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |    `f1` |   `f2` |
 |:----------------------|:--------|--------------:|--------:|--------:|---------:|------:|--------:|-------:|
 | **NEGmir~any_better** | better  |     14,076.00 |    4.44 |    0.75 | 1,148.18 |   381 | 289,770 |    416 |
 | **NEGmir~any_easier** | easier  |      2,409.00 |    2.42 |    0.75 |   181.98 |    61 | 289,770 |     67 |
 | **NEGmir~any_worse**  | worse   |      8,490.00 |    2.87 |    0.72 |   248.63 |    88 | 289,770 |    100 |
 | **NEGmir~any_closer** | closer  |        986.00 |    2.21 |    0.68 |   149.62 |    56 | 289,770 |     66 |


#### 6. _ever_


Top 5 `RBdirect` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                         | `adj`   |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |      `f1` |   `f2` |
 |:------------------------|:--------|--------------:|--------:|--------:|---------:|------:|----------:|-------:|
 | **NEGany~ever_simple**  | simple  |    427,167.00 |    5.54 |    0.77 | 1,142.04 |   212 | 3,226,213 |    262 |
 | **NEGany~ever_easy**    | easy    |    771,307.00 |    5.06 |    0.63 | 2,030.58 |   430 | 3,226,213 |    641 |
 | **NEGany~ever_good**    | good    |  2,037,285.00 |    3.76 |    0.40 | 1,178.00 |   332 | 3,226,213 |    756 |
 | **NEGany~ever_perfect** | perfect |    164,519.00 |    3.48 |    0.37 |   736.05 |   217 | 3,226,213 |    527 |
 | **NEGany~ever_able**    | able    |    428,268.00 |    1.81 |    0.13 |   363.95 |   234 | 3,226,213 |  1,398 |


Top 6 `NEGmirror` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                         | `adj`   |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |    `f1` |   `f2` |
 |:------------------------|:--------|--------------:|--------:|--------:|---------:|------:|--------:|-------:|
 | **NEGmir~ever_easy**    | easy    |     20,050.00 |    3.21 |    0.83 | 1,311.83 |   367 | 289,770 |    368 |
 | **NEGmir~ever_perfect** | perfect |      3,708.00 |    2.38 |    0.83 |   735.10 |   207 | 289,770 |    208 |
 | **NEGmir~ever_good**    | good    |     33,540.00 |    4.72 |    0.82 | 1,034.95 |   298 | 289,770 |    302 |
 | **NEGmir~ever_wrong**   | wrong   |     20,866.00 |    2.56 |    0.82 |   349.21 |   102 | 289,770 |    104 |
 | **NEGmir~ever_free**    | free    |      5,043.00 |    1.97 |    0.81 |   231.61 |    69 | 289,770 |     71 |
 | **NEGmir~ever_able**    | able    |      6,448.00 |    3.66 |    0.79 |   437.65 |   136 | 289,770 |    143 |


#### 7. _yet_


Top 10 `RBdirect` "yet_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |      `G2` |    `f` |      `f1` |   `f2` |
 |:-------------------------|:----------|--------------:|--------:|--------:|----------:|-------:|----------:|-------:|
 | **NEGany~yet_clear**     | clear     |    491,108.00 |   10.26 |    0.95 | 67,924.56 | 10,553 | 3,226,213 | 10,693 |
 | **NEGany~yet_eligible**  | eligible  |     49,578.00 |    7.72 |    0.94 |  2,929.15 |    459 | 3,226,213 |    468 |
 | **NEGany~yet_official**  | official  |      9,778.00 |    7.33 |    0.94 |  2,236.98 |    353 | 3,226,213 |    362 |
 | **NEGany~yet_ready**     | ready     |    240,297.00 |    9.23 |    0.93 | 48,012.06 |  7,611 | 3,226,213 |  7,838 |
 | **NEGany~yet_certain**   | certain   |    104,544.00 |    8.12 |    0.93 |  5,491.41 |    874 | 3,226,213 |    903 |
 | **NEGany~yet_complete**  | complete  |    107,018.00 |    8.42 |    0.92 | 13,815.99 |  2,220 | 3,226,213 |  2,314 |
 | **NEGany~yet_sure**      | sure      |    844,981.00 |    8.37 |    0.92 | 12,379.79 |  1,990 | 3,226,213 |  2,075 |
 | **NEGany~yet_available** | available |    866,272.00 |    7.69 |    0.87 | 44,196.15 |  7,481 | 3,226,213 |  8,238 |
 | **NEGany~yet_right**     | right     |    204,572.00 |    6.50 |    0.92 |  1,254.20 |    202 | 3,226,213 |    211 |
 | **NEGany~yet_final**     | final     |      9,657.00 |    7.45 |    0.91 |  4,028.75 |    659 | 3,226,213 |    699 |

 No bigrams found in loaded `NEGmirror` AM table.

#### 8. _longer_


Top 5 `RBdirect` "longer_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `G2` |    `f` |       `f1` |   `f2` |
 |:-------------------------|:----------|--------------:|--------:|--------:|-------:|-------:|-----------:|-------:|
 | **COM~longer_lasting**   | lasting   |     24,344.00 |    1.44 |    0.04 | 244.09 |  3,860 | 83,102,035 |  3,866 |
 | **COM~longer_enough**    | enough    |    453,790.00 |    1.41 |    0.03 | 216.98 |  3,952 | 83,102,035 |  3,964 |
 | **COM~longer_able**      | able      |    428,268.00 |    2.28 |    0.03 | 623.67 | 11,677 | 83,102,035 | 11,716 |
 | **COM~longer_available** | available |    866,272.00 |    2.45 |    0.03 | 974.55 | 18,865 | 83,102,035 | 18,935 |
 | **COM~longer_necessary** | necessary |    187,396.00 |    1.27 |    0.03 | 220.07 |  5,365 | 83,102,035 |  5,399 |

 No bigrams found in loaded `NEGmirror` AM table.

#### 9. _immediately_


Top 5 `RBdirect` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                                  | `adj`     |   `adj_total` |   `LRC` |   `dP1` |       `G2` |    `f` |      `f1` |   `f2` |
 |:---------------------------------|:----------|--------------:|--------:|--------:|-----------:|-------:|----------:|-------:|
 | **NEGany~immediately_possible**  | possible  |    364,265.00 |    7.68 |    0.90 |   6,269.26 |  1,027 | 3,226,213 |  1,091 |
 | **NEGany~immediately_clear**     | clear     |    491,108.00 |    8.32 |    0.90 | 153,302.22 | 25,276 | 3,226,213 | 27,066 |
 | **NEGany~immediately_available** | available |    866,272.00 |    5.77 |    0.66 | 102,962.94 | 21,297 | 3,226,213 | 30,725 |
 | **NEGany~immediately_able**      | able      |    428,268.00 |    4.87 |    0.58 |   2,851.84 |    639 | 3,226,213 |  1,036 |
 | **NEGany~immediately_obvious**   | obvious   |    193,498.00 |    4.59 |    0.49 |   9,043.23 |  2,258 | 3,226,213 |  4,305 |


Top 1 `NEGmirror` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                                  | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `G2` |   `f` |    `f1` |   `f2` |
 |:---------------------------------|:----------|--------------:|--------:|--------:|-------:|------:|--------:|-------:|
 | **NEGmir~immediately_available** | available |     12,636.00 |    1.94 |    0.43 | 254.47 |   162 | 289,770 |    274 |



In [6]:
# %%

bigram_dfs['RBdirect'].filter(like='~before_', axis=0)

,l2,f,E11,am_log_likelihood,...,adv_total,adj_total,l1,first_char
key,,,,,,,,,
COM~before_dark,before_dark,72,69.82,2.12,...,681,72272,COMPLEMENT,b
NEGany~before_available,before_available,177,8.23,"1,020.91",...,681,666909,NEGATED,b


In [7]:
# %%

for key, info in samples_dict.items():
    if key in ('bigrams', 'adj'):
        key = f'ALL {key.replace("adj", "adjectives")}'
    formatted_iter = [
        f'_{a.replace("_", " ")}_' for a
        in (info['adj'] if isinstance(info, dict)
            else info)]
    print_iter(formatted_iter,
               header=f'1. _{key}_ ({len(formatted_iter)} unique)',
               bullet='1.', indent=3)


1. _necessarily_ (16 unique)
   1. _useful_
   1. _reliable_
   1. _essential_
   1. _proud_
   1. _wrong_
   1. _representative_
   1. _fun_
   1. _illegal_
   1. _easy_
   1. _right_
   1. _new_
   1. _true_
   1. _better_
   1. _surprising_
   1. _bad_
   1. _indicative_

1. _that_ (20 unique)
   1. _clear_
   1. _far-fetched_
   1. _uncommon_
   1. _complicated_
   1. _unusual_
   1. _familiar_
   1. _popular_
   1. _keen_
   1. _great_
   1. _surprising_
   1. _hard_
   1. _thrilled_
   1. _fond_
   1. _noticeable_
   1. _easy_
   1. _dissimilar_
   1. _simple_
   1. _big_
   1. _impressive_
   1. _comfortable_

1. _exactly_ (11 unique)
   1. _famous_
   1. _clear_
   1. _surprising_
   1. _easy_
   1. _sure_
   1. _new_
   1. _shocking_
   1. _pleasant_
   1. _conducive_
   1. _difficult_
   1. _cheap_

1. _any_ (16 unique)
   1. _smarter_
   1. _higher_
   1. _nicer_
   1. _clearer_
   1. _brighter_
   1. _good_
   1. _closer_
   1. _cheaper_
   1. _different_
   1. _simpler_
 


1. _necessarily_ (16 unique)
   1. _fun_
   1. _useful_
   1. _reliable_
   1. _proud_
   1. _easy_
   1. _right_
   1. _representative_
   1. _better_
   1. _surprising_
   1. _essential_
   1. _true_
   1. _wrong_
   1. _illegal_
   1. _indicative_
   1. _new_
   1. _bad_

1. _that_ (20 unique)
   1. _comfortable_
   1. _great_
   1. _thrilled_
   1. _clear_
   1. _fond_
   1. _popular_
   1. _dissimilar_
   1. _unusual_
   1. _surprising_
   1. _simple_
   1. _keen_
   1. _familiar_
   1. _complicated_
   1. _far-fetched_
   1. _uncommon_
   1. _easy_
   1. _big_
   1. _noticeable_
   1. _impressive_
   1. _hard_

1. _exactly_ (11 unique)
   1. _difficult_
   1. _cheap_
   1. _easy_
   1. _conducive_
   1. _surprising_
   1. _shocking_
   1. _pleasant_
   1. _famous_
   1. _sure_
   1. _clear_
   1. _new_

1. _any_ (16 unique)
   1. _brighter_
   1. _younger_
   1. _different_
   1. _cheaper_
   1. _better_
   1. _simpler_
   1. _smarter_
   1. _happier_
   1. _higher_
   1. _good_
   1. _closer_
   1. _worse_
   1. _nicer_
   1. _clearer_
   1. _bigger_
   1. _easier_

1. _remotely_ (14 unique)
   1. _close_
   1. _believable_
   1. _interesting_
   1. _similar_
   1. _interested_
   1. _possible_
   1. _surprising_
   1. _accurate_
   1. _ready_
   1. _funny_
   1. _qualified_
   1. _true_
   1. _comparable_
   1. _new_

1. _ever_ (13 unique)
   1. _certain_
   1. _able_
   1. _easy_
   1. _perfect_
   1. _sick_
   1. _enough_
   1. _simple_
   1. _sure_
   1. _good_
   1. _boring_
   1. _closer_
   1. _black_
   1. _satisfied_

1. _yet_ (13 unique)
   1. _certain_
   1. _over_
   1. _final_
   1. _available_
   1. _mainstream_
   1. _convinced_
   1. _ready_
   1. _sure_
   1. _official_
   1. _online_
   1. _eligible_
   1. _complete_
   1. _clear_

1. _immediately_ (10 unique)
   1. _certain_
   1. _able_
   1. _available_
   1. _obvious_
   1. _intuitive_
   1. _forthcoming_
   1. _sure_
   1. _clear_
   1. _reachable_
   1. _possible_

1. _particularly_ (17 unique)
   1. _close_
   1. _athletic_
   1. _revolutionary_
   1. _remarkable_
   1. _comfortable_
   1. _surprising_
   1. _original_
   1. _forthcoming_
   1. _radical_
   1. _flashy_
   1. _religious_
   1. _novel_
   1. _wrong_
   1. _fast_
   1. _likeable_
   1. _likable_
   1. _new_

1. _inherently_ (9 unique)
   1. _problematic_
   1. _better_
   1. _negative_
   1. _improper_
   1. _good_
   1. _bad_
   1. _illegal_
   1. _wrong_
   1. _evil_

1. _terribly_ (17 unique)
   1. _different_
   1. _likely_
   1. _uncommon_
   1. _interesting_
   1. _reliable_
   1. _remarkable_
   1. _productive_
   1. _interested_
   1. _unusual_
   1. _surprising_
   1. _original_
   1. _special_
   1. _sure_
   1. _clear_
   1. _impressive_
   1. _popular_
   1. _new_

1. _ALL bigrams_ (156 unique)
   1. _particularly wrong_
   1. _inherently wrong_
   1. _that easy_
   1. _that popular_
   1. _exactly surprising_
   1. _any brighter_
   1. _terribly unusual_
   1. _ever sick_
   1. _exactly clear_
   1. _any good_
   1. _ever easy_
   1. _immediately sure_
   1. _remotely qualified_
   1. _inherently good_
   1. _particularly athletic_
   1. _terribly productive_
   1. _necessarily reliable_
   1. _any worse_
   1. _remotely new_
   1. _remotely close_
   1. _exactly conducive_
   1. _ever certain_
   1. _that familiar_
   1. _necessarily better_
   1. _exactly sure_
   1. _ever able_
   1. _inherently illegal_
   1. _particularly remarkable_
   1. _particularly close_
   1. _any clearer_
   1. _terribly remarkable_
   1. _that simple_
   1. _any nicer_
   1. _yet over_
   1. _yet sure_
   1. _that comfortable_
   1. _necessarily proud_
   1. _necessarily surprising_
   1. _immediately available_
   1. _exactly famous_
   1. _particularly comfortable_
   1. _exactly cheap_
   1. _exactly pleasant_
   1. _necessarily fun_
   1. _particularly radical_
   1. _any easier_
   1. _yet certain_
   1. _yet complete_
   1. _necessarily easy_
   1. _exactly difficult_
   1. _ever closer_
   1. _yet eligible_
   1. _exactly shocking_
   1. _terribly sure_
   1. _yet clear_
   1. _remotely funny_
   1. _remotely interesting_
   1. _that impressive_
   1. _particularly fast_
   1. _that hard_
   1. _necessarily useful_
   1. _ever satisfied_
   1. _necessarily wrong_
   1. _immediately clear_
   1. _any higher_
   1. _ever enough_
   1. _immediately possible_
   1. _that noticeable_
   1. _terribly surprising_
   1. _terribly likely_
   1. _remotely believable_
   1. _terribly clear_
   1. _inherently problematic_
   1. _terribly uncommon_
   1. _ever black_
   1. _necessarily true_
   1. _terribly interested_
   1. _that dissimilar_
   1. _necessarily indicative_
   1. _any simpler_
   1. _immediately certain_
   1. _inherently bad_
   1. _exactly easy_
   1. _any smarter_
   1. _yet available_
   1. _any happier_
   1. _any closer_
   1. _particularly original_
   1. _particularly religious_
   1. _yet online_
   1. _any bigger_
   1. _that thrilled_
   1. _yet convinced_
   1. _particularly likeable_
   1. _that great_
   1. _that unusual_
   1. _any cheaper_
   1. _yet official_
   1. _terribly original_
   1. _yet final_
   1. _ever sure_
   1. _remotely possible_
   1. _particularly surprising_
   1. _that far-fetched_
   1. _necessarily representative_
   1. _that surprising_
   1. _ever perfect_
   1. _remotely similar_
   1. _inherently improper_
   1. _that fond_
   1. _ever good_
   1. _remotely true_
   1. _terribly reliable_
   1. _remotely interested_
   1. _terribly new_
   1. _any younger_
   1. _terribly different_
   1. _that complicated_
   1. _immediately obvious_
   1. _particularly likable_
   1. _immediately intuitive_
   1. _necessarily new_
   1. _yet ready_
   1. _particularly flashy_
   1. _terribly special_
   1. _particularly forthcoming_
   1. _that big_
   1. _immediately reachable_
   1. _immediately able_
   1. _necessarily bad_
   1. _terribly popular_
   1. _inherently better_
   1. _any better_
   1. _remotely comparable_
   1. _ever boring_
   1. _particularly new_
   1. _terribly interesting_
   1. _exactly new_
   1. _that keen_
   1. _yet mainstream_
   1. _inherently negative_
   1. _that clear_
   1. _that uncommon_
   1. _necessarily right_
   1. _any different_
   1. _inherently evil_
   1. _particularly revolutionary_
   1. _ever simple_
   1. _terribly impressive_
   1. _necessarily essential_
   1. _particularly novel_
   1. _necessarily illegal_
   1. _immediately forthcoming_
   1. _remotely surprising_
   1. _remotely accurate_
   1. _remotely ready_

1. _ALL adjectives_ (106 unique)
   1. _close_
   1. _proud_
   1. _interested_
   1. _radical_
   1. _true_
   1. _different_
   1. _fond_
   1. _likely_
   1. _cheaper_
   1. _similar_
   1. _dissimilar_
   1. _interesting_
   1. _ready_
   1. _online_
   1. _difficult_
   1. _better_
   1. _representative_
   1. _sure_
   1. _eligible_
   1. _impressive_
   1. _possible_
   1. _evil_
   1. _right_
   1. _essential_
   1. _convinced_
   1. _flashy_
   1. _special_
   1. _simple_
   1. _religious_
   1. _keen_
   1. _complete_
   1. _likable_
   1. _complicated_
   1. _accurate_
   1. _far-fetched_
   1. _enough_
   1. _illegal_
   1. _likeable_
   1. _bigger_
   1. _easier_
   1. _younger_
   1. _available_
   1. _big_
   1. _negative_
   1. _simpler_
   1. _thrilled_
   1. _comparable_
   1. _brighter_
   1. _revolutionary_
   1. _productive_
   1. _comfortable_
   1. _great_
   1. _improper_
   1. _clear_
   1. _reachable_
   1. _intuitive_
   1. _certain_
   1. _obvious_
   1. _perfect_
   1. _unusual_
   1. _happier_
   1. _pleasant_
   1. _worse_
   1. _novel_
   1. _able_
   1. _familiar_
   1. _useful_
   1. _athletic_
   1. _sick_
   1. _funny_
   1. _qualified_
   1. _mainstream_
   1. _closer_
   1. _problematic_
   1. _fun_
   1. _uncommon_
   1. _final_
   1. _cheap_
   1. _easy_
   1. _forthcoming_
   1. _famous_
   1. _hard_
   1. _over_
   1. _conducive_
   1. _smarter_
   1. _original_
   1. _fast_
   1. _bad_
   1. _satisfied_
   1. _surprising_
   1. _good_
   1. _indicative_
   1. _nicer_
   1. _clearer_
   1. _higher_
   1. _new_
   1. _black_
   1. _shocking_
   1. _remarkable_
   1. _wrong_
   1. _reliable_
   1. _noticeable_
   1. _official_
   1. _boring_
   1. _popular_
   1. _believable_


In [8]:
NEG_bigrams_sample = pd.concat(
    (ad['both'] for ad in samples_dict.values() if isinstance(ad, dict))).sort_values('LRC', ascending=False)

In [12]:
top_NEGbigram_df_path =  OUT_DIR.joinpath(
    f'{TAG}-Top{K}_NEG-ADV-{ADV_FLOOR}_Top{bigram_k}-bigrams-{bigram_floor}.{timestamp_today()}.csv')
print(f'> Saved as:  \n>   `{top_NEGbigram_df_path}`')

NEG_bigrams_sample.to_csv(top_NEGbigram_df_path)
nb_show_table(NEG_bigrams_sample.filter(adjust_ams(FOCUS)
                                        ).sort_values('LRC', ascending=False),
              outpath=top_NEGbigram_df_path.with_suffix('.md'))

> Saved as:
>   `/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8_NEG-ADV-5000_Top10-bigrams-50.2024-07-29.csv`

|                                       |    `f` |   `dP1` |   `P1` |   `LRC` |       `G2` |   `MI` |   `odds_r_disc` |    `t` |        `N` |       `f1` |   `f2` |   `exp_f` |   `unexp_f` | `l1`       | `l2`                       | `adv`        |   `adv_total` | `adj`          |   `adj_total` |
|:--------------------------------------|-------:|--------:|-------:|--------:|-----------:|-------:|----------------:|-------:|-----------:|-----------:|-------:|----------:|------------:|:-----------|:---------------------------|:-------------|--------------:|:---------------|--------------:|
| **NEGany~yet_clear**                  | 10,406 |    0.95 |   0.99 |   10.77 |  64,409.97 |   1.36 |            3.51 |  97.54 | 72,839,589 |  3,173,660 | 10,476 |    456.44 |    9,949.56 | NEGATED    | yet_clear                  | yet          |        95,763 | clear        

> Saved as:  
>   `/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8_NEG-ADV-5000_Top10-bigrams-50.2024-07-29.csv`

|                                       |    `f` |   `dP1` |   `P1` |   `LRC` |       `G2` |   `MI` |   `odds_r_disc` |    `t` |        `N` |       `f1` |   `f2` |   `exp_f` |   `unexp_f` | `l1`       | `l2`                       | `adv`        |   `adv_total` | `adj`          |   `adj_total` |
|:--------------------------------------|-------:|--------:|-------:|--------:|-----------:|-------:|----------------:|-------:|-----------:|-----------:|-------:|----------:|------------:|:-----------|:---------------------------|:-------------|--------------:|:---------------|--------------:|
| **NEGany~yet_clear**                  | 10,406 |    0.95 |   0.99 |   10.77 |  64,409.97 |   1.36 |            3.51 |  97.54 | 72,839,589 |  3,173,660 | 10,476 |    456.44 |    9,949.56 | NEGATED    | yet_clear                  | yet          |        95,763 | clear          |       349,214 |
| **NEGany~yet_ready**                  |  7,501 |    0.94 |   0.99 |    9.93 |  45,985.07 |   1.36 |            3.22 |  82.79 | 72,839,589 |  3,173,660 |  7,599 |    331.09 |    7,169.91 | NEGATED    | yet_ready                  | yet          |        95,763 | ready          |       141,590 |
| **NEGany~necessarily_indicative**     |  1,389 |    0.95 |   0.99 |    9.43 |   8,577.54 |   1.36 |            3.42 |  35.63 | 72,839,589 |  3,173,660 |  1,400 |     61.00 |    1,328.00 | NEGATED    | necessarily_indicative     | necessarily  |        48,947 | indicative     |         8,148 |
| **NEGany~that_uncommon**              |    802 |    0.95 |   1.00 |    9.43 |   4,998.32 |   1.36 |            3.85 |  27.08 | 72,839,589 |  3,173,660 |    804 |     35.03 |      766.97 | NEGATED    | that_uncommon              | that         |       208,262 | uncommon       |        11,312 |
| **NEGany~exactly_easy**               |  1,066 |    0.95 |   0.99 |    9.32 |   6,596.91 |   1.36 |            3.49 |  31.22 | 72,839,589 |  3,173,660 |  1,073 |     46.75 |    1,019.25 | NEGATED    | exactly_easy               | exactly      |        58,643 | easy           |       579,827 |
| **NEGany~yet_complete**               |  2,174 |    0.94 |   0.98 |    9.20 |  13,277.09 |   1.35 |            3.14 |  44.56 | 72,839,589 |  3,173,660 |  2,208 |     96.20 |    2,077.80 | NEGATED    | yet_complete               | yet          |        95,763 | complete       |        86,361 |
| **NEGany~that_surprising**            |  1,133 |    0.95 |   0.99 |    9.20 |   6,986.81 |   1.36 |            3.37 |  32.18 | 72,839,589 |  3,173,660 |  1,143 |     49.80 |    1,083.20 | NEGATED    | that_surprising            | that         |       208,262 | surprising     |        70,540 |
| **NEGany~exactly_new**                |  1,371 |    0.94 |   0.99 |    9.10 |   8,410.32 |   1.36 |            3.24 |  35.39 | 72,839,589 |  3,173,660 |  1,388 |     60.48 |    1,310.52 | NEGATED    | exactly_new                | exactly      |        58,643 | new            |       253,862 |
| **NEGany~necessarily_easy**           |    909 |    0.95 |   0.99 |    9.01 |   5,605.64 |   1.36 |            3.37 |  28.82 | 72,839,589 |  3,173,660 |    917 |     39.95 |      869.05 | NEGATED    | necessarily_easy           | necessarily  |        48,947 | easy           |       579,827 |
| **NEGany~yet_final**                  |    640 |    0.95 |   1.00 |    8.97 |   3,972.92 |   1.36 |            3.60 |  24.19 | 72,839,589 |  3,173,660 |    643 |     28.02 |      611.98 | NEGATED    | yet_final                  | yet          |        95,763 | final          |         5,860 |
| **NEGany~exactly_cheap**              |    691 |    0.95 |   0.99 |    8.96 |   4,281.59 |   1.36 |            3.53 |  25.13 | 72,839,589 |  3,173,660 |    695 |     30.28 |      660.72 | NEGATED    | exactly_cheap              | exactly      |        58,643 | cheap          |        60,531 |
| **NEGany~yet_eligible**               |    448 |    0.96 |   1.00 |    8.96 |   2,807.56 |   1.36 |            4.29 |  20.24 | 72,839,589 |  3,173,660 |    448 |     19.52 |      428.48 | NEGATED    | yet_eligible               | yet          |        95,763 | eligible       |        23,252 |
| **NEGany~that_unusual**               |    977 |    0.95 |   0.99 |    8.92 |   6,003.05 |   1.36 |            3.27 |  29.88 | 72,839,589 |  3,173,660 |    988 |     43.05 |      933.95 | NEGATED    | that_unusual               | that         |       208,262 | unusual        |        71,234 |
| **NEGany~exactly_clear**              |  1,746 |    0.94 |   0.98 |    8.78 |  10,578.33 |   1.35 |            3.00 |  39.92 | 72,839,589 |  3,173,660 |  1,784 |     77.73 |    1,668.27 | NEGATED    | exactly_clear              | exactly      |        58,643 | clear          |       349,214 |
| **NEGany~exactly_surprising**         |    440 |    0.95 |   1.00 |    8.71 |   2,743.34 |   1.36 |            3.81 |  20.06 | 72,839,589 |  3,173,660 |    441 |     19.21 |      420.79 | NEGATED    | exactly_surprising         | exactly      |        58,643 | surprising     |        70,540 |
| **NEGany~that_complicated**           |  1,207 |    0.94 |   0.98 |    8.68 |   7,337.84 |   1.35 |            3.05 |  33.20 | 72,839,589 |  3,173,660 |  1,230 |     53.59 |    1,153.41 | NEGATED    | that_complicated           | that         |       208,262 | complicated    |       159,822 |
| **NEGany~terribly_surprising**        |    949 |    0.94 |   0.98 |    8.66 |   5,794.10 |   1.35 |            3.13 |  29.44 | 72,839,589 |  3,173,660 |    964 |     42.00 |      907.00 | NEGATED    | terribly_surprising        | terribly     |        58,964 | surprising     |        70,540 |
| **NEGany~yet_official**               |    352 |    0.96 |   1.00 |    8.61 |   2,205.93 |   1.36 |            4.19 |  17.94 | 72,839,589 |  3,173,660 |    352 |     15.34 |      336.66 | NEGATED    | yet_official               | yet          |        95,763 | official       |         6,853 |
| **NEGany~that_hard**                  |  9,948 |    0.91 |   0.96 |    8.59 |  58,817.24 |   1.34 |            2.70 |  95.21 | 72,839,589 |  3,173,660 | 10,380 |    452.26 |    9,495.74 | NEGATED    | that_hard                  | that         |       208,262 | hard           |       348,463 |
| **NEGany~that_familiar**              |  1,126 |    0.93 |   0.97 |    8.35 |   6,781.11 |   1.35 |            2.91 |  32.05 | 72,839,589 |  3,173,660 |  1,156 |     50.37 |    1,075.63 | NEGATED    | that_familiar              | that         |       208,262 | familiar       |       156,296 |
| **NEGany~necessarily_representative** |    487 |    0.95 |   0.99 |    8.34 |   2,996.58 |   1.36 |            3.29 |  21.10 | 72,839,589 |  3,173,660 |    492 |     21.44 |      465.56 | NEGATED    | necessarily_representative | necessarily  |        48,947 | representative |        18,355 |
| **NEGany~necessarily_surprising**     |    340 |    0.95 |   1.00 |    8.33 |   2,117.16 |   1.36 |            3.70 |  17.63 | 72,839,589 |  3,173,660 |    341 |     14.86 |      325.14 | NEGATED    | necessarily_surprising     | necessarily  |        48,947 | surprising     |        70,540 |
| **NEGany~immediately_clear**          | 24,416 |    0.89 |   0.94 |    8.16 | 141,186.69 |   1.33 |            2.52 | 149.00 | 72,839,589 |  3,173,660 | 26,038 |  1,134.49 |   23,281.51 | NEGATED    | immediately_clear          | immediately  |        96,973 | clear          |       349,214 |
| **NEGany~necessarily_new**            |    482 |    0.94 |   0.98 |    7.94 |   2,923.82 |   1.35 |            3.00 |  20.98 | 72,839,589 |  3,173,660 |    492 |     21.44 |      460.56 | NEGATED    | necessarily_new            | necessarily  |        48,947 | new            |       253,862 |
| **NEGany~that_dissimilar**            |    304 |    0.95 |   0.99 |    7.86 |   1,871.65 |   1.36 |            3.28 |  16.67 | 72,839,589 |  3,173,660 |    307 |     13.38 |      290.62 | NEGATED    | that_dissimilar            | that         |       208,262 | dissimilar     |         4,605 |
| **NEGany~exactly_conducive**          |    208 |    0.96 |   1.00 |    7.82 |   1,303.50 |   1.36 |            3.96 |  13.79 | 72,839,589 |  3,173,660 |    208 |      9.06 |      198.94 | NEGATED    | exactly_conducive          | exactly      |        58,643 | conducive      |         9,110 |
| **NEGany~that_noticeable**            |    264 |    0.95 |   0.99 |    7.65 |   1,621.81 |   1.36 |            3.22 |  15.53 | 72,839,589 |  3,173,660 |    267 |     11.63 |      252.37 | NEGATED    | that_noticeable            | that         |       208,262 | noticeable     |        31,467 |
| **NEGany~immediately_possible**       |  1,000 |    0.91 |   0.95 |    7.62 |   5,845.77 |   1.34 |            2.61 |  30.17 | 72,839,589 |  3,173,660 |  1,054 |     45.92 |      954.08 | NEGATED    | immediately_possible       | immediately  |        96,973 | possible       |       245,272 |
| **NEGany~yet_convinced**              |    169 |    0.96 |   1.00 |    7.50 |   1,059.09 |   1.36 |            3.87 |  12.43 | 72,839,589 |  3,173,660 |    169 |      7.36 |      161.64 | NEGATED    | yet_convinced              | yet          |        95,763 | convinced      |        12,132 |
| **NEGany~exactly_shocking**           |    151 |    0.96 |   1.00 |    7.33 |     946.29 |   1.36 |            3.82 |  11.75 | 72,839,589 |  3,173,660 |    151 |      6.58 |      144.42 | NEGATED    | exactly_shocking           | exactly      |        58,643 | shocking       |        35,115 |
| **NEGany~exactly_pleasant**           |    142 |    0.96 |   1.00 |    7.24 |     889.88 |   1.36 |            3.80 |  11.40 | 72,839,589 |  3,173,660 |    142 |      6.19 |      135.81 | NEGATED    | exactly_pleasant           | exactly      |        58,643 | pleasant       |        52,223 |
| **NEGany~yet_over**                   |    162 |    0.95 |   0.99 |    7.21 |   1,003.13 |   1.36 |            3.38 |  12.17 | 72,839,589 |  3,173,660 |    163 |      7.10 |      154.90 | NEGATED    | yet_over                   | yet          |        95,763 | over           |         3,983 |
| **NEGany~exactly_famous**             |    130 |    0.96 |   1.00 |    7.10 |     814.68 |   1.36 |            3.76 |  10.90 | 72,839,589 |  3,173,660 |    130 |      5.66 |      124.34 | NEGATED    | exactly_famous             | exactly      |        58,643 | famous         |       223,813 |
| **NEGany~exactly_difficult**          |    126 |    0.96 |   1.00 |    7.05 |     789.62 |   1.36 |            3.74 |  10.74 | 72,839,589 |  3,173,660 |    126 |      5.49 |      120.51 | NEGATED    | exactly_difficult          | exactly      |        58,643 | difficult      |       732,106 |
| **NEGany~necessarily_useful**         |    104 |    0.96 |   1.00 |    6.75 |     651.75 |   1.36 |            3.66 |   9.75 | 72,839,589 |  3,173,660 |    104 |      4.53 |       99.47 | NEGATED    | necessarily_useful         | necessarily  |        48,947 | useful         |       227,709 |
| **NEGany~yet_online**                 |     98 |    0.96 |   1.00 |    6.66 |     614.14 |   1.36 |            3.64 |   9.47 | 72,839,589 |  3,173,660 |     98 |      4.27 |       93.73 | NEGATED    | yet_online                 | yet          |        95,763 | online         |        15,650 |
| **NEGany~necessarily_fun**            |     96 |    0.96 |   1.00 |    6.62 |     601.61 |   1.36 |            3.63 |   9.37 | 72,839,589 |  3,173,660 |     96 |      4.18 |       91.82 | NEGATED    | necessarily_fun            | necessarily  |        48,947 | fun            |       190,026 |
| **NEGany~necessarily_essential**      |     93 |    0.96 |   1.00 |    6.57 |     582.81 |   1.36 |            3.61 |   9.22 | 72,839,589 |  3,173,660 |     93 |      4.05 |       88.95 | NEGATED    | necessarily_essential      | necessarily  |        48,947 | essential      |        69,845 |
| **NEGmir~ever_easy**                  |    368 |    0.83 |   1.00 |    6.44 |   1,285.01 |   0.76 |            3.08 |  15.89 |  1,701,929 |    291,732 |    369 |     63.25 |      304.75 | NEGMIR     | ever_easy                  | ever         |         5,060 | easy           |        18,610 |
| **NEGany~terribly_original**          |    199 |    0.90 |   0.94 |    6.41 |   1,150.48 |   1.33 |            2.51 |  13.45 | 72,839,589 |  3,173,660 |    212 |      9.24 |      189.76 | NEGATED    | terribly_original          | terribly     |        58,964 | original       |        37,594 |
| **NEGany~necessarily_reliable**       |     81 |    0.96 |   1.00 |    6.35 |     507.61 |   1.36 |            3.55 |   8.61 | 72,839,589 |  3,173,660 |     81 |      3.53 |       77.47 | NEGATED    | necessarily_reliable       | necessarily  |        48,947 | reliable       |        90,598 |
| **NEGany~any_happier**                |    828 |    0.82 |   0.86 |    6.34 |   4,420.49 |   1.30 |            2.13 |  27.32 | 72,839,589 |  3,173,660 |    963 |     41.96 |      786.04 | NEGATED    | any_happier                | any          |        34,382 | happier        |        16,606 |
| **NEGany~terribly_different**         |    366 |    0.85 |   0.89 |    6.33 |   2,022.47 |   1.31 |            2.27 |  18.20 | 72,839,589 |  3,173,660 |    409 |     17.82 |      348.18 | NEGATED    | terribly_different         | terribly     |        58,964 | different      |       825,838 |
| **NEGany~terribly_uncommon**          |    103 |    0.94 |   0.98 |    6.33 |     625.85 |   1.35 |            2.96 |   9.70 | 72,839,589 |  3,173,660 |    105 |      4.57 |       98.43 | NEGATED    | terribly_uncommon          | terribly     |        58,964 | uncommon       |        11,312 |
| **NEGany~terribly_likely**            |    108 |    0.93 |   0.97 |    6.26 |     649.50 |   1.35 |            2.83 |   9.93 | 72,839,589 |  3,173,660 |    111 |      4.84 |      103.16 | NEGATED    | terribly_likely            | terribly     |        58,964 | likely         |       890,421 |
| **NEGany~terribly_unusual**           |    146 |    0.90 |   0.94 |    6.17 |     847.05 |   1.33 |            2.53 |  11.52 | 72,839,589 |  3,173,660 |    155 |      6.75 |      139.25 | NEGATED    | terribly_unusual           | terribly     |        58,964 | unusual        |        71,234 |
| **NEGany~necessarily_proud**          |     71 |    0.96 |   1.00 |    6.14 |     444.94 |   1.36 |            3.50 |   8.06 | 72,839,589 |  3,173,660 |     71 |      3.09 |       67.91 | NEGATED    | necessarily_proud          | necessarily  |        48,947 | proud          |       207,536 |
| **NEGany~particularly_religious**     |    485 |    0.81 |   0.86 |    6.12 |   2,585.71 |   1.29 |            2.12 |  20.90 | 72,839,589 |  3,173,660 |    565 |     24.62 |      460.38 | NEGATED    | particularly_religious     | particularly |       513,668 | religious      |        28,028 |
| **NEGany~yet_mainstream**             |     70 |    0.96 |   1.00 |    6.11 |     438.67 |   1.36 |            3.49 |   8.00 | 72,839,589 |  3,173,660 |     70 |      3.05 |       66.95 | NEGATED    | yet_mainstream             | yet          |        95,763 | mainstream     |        17,792 |
| **NEGany~particularly_new**           |    747 |    0.79 |   0.83 |    6.04 |   3,874.46 |   1.28 |            2.03 |  25.90 | 72,839,589 |  3,173,660 |    900 |     39.21 |      707.79 | NEGATED    | particularly_new           | particularly |       513,668 | new            |       253,862 |
| **NEGany~terribly_impressive**        |    196 |    0.86 |   0.90 |    5.97 |   1,087.64 |   1.31 |            2.28 |  13.32 | 72,839,589 |  3,173,660 |    218 |      9.50 |      186.50 | NEGATED    | terribly_impressive        | terribly     |        58,964 | impressive     |       178,051 |
| **NEGmir~particularly_new**           |    404 |    0.81 |   0.99 |    5.92 |   1,365.17 |   0.76 |            2.48 |  16.60 |  1,701,929 |    291,732 |    410 |     70.28 |      333.72 | NEGMIR     | particularly_new           | particularly |        13,003 | new            |        12,836 |
| **NEGany~particularly_original**      |    360 |    0.80 |   0.85 |    5.86 |   1,894.60 |   1.29 |            2.08 |  18.00 | 72,839,589 |  3,173,660 |    426 |     18.56 |      341.44 | NEGATED    | particularly_original      | particularly |       513,668 | original       |        37,594 |
| **NEGany~that_far-fetched**           |     59 |    0.96 |   1.00 |    5.83 |     369.74 |   1.36 |            3.42 |   7.35 | 72,839,589 |  3,173,660 |     59 |      2.57 |       56.43 | NEGATED    | that_far-fetched           | that         |       208,262 | far-fetched    |         5,185 |
| **NEGany~that_thrilled**              |     59 |    0.96 |   1.00 |    5.83 |     369.74 |   1.36 |            3.42 |   7.35 | 72,839,589 |  3,173,660 |     59 |      2.57 |       56.43 | NEGATED    | that_thrilled              | that         |       208,262 | thrilled       |        24,182 |
| **NEGmir~ever_simple**                |    206 |    0.83 |   1.00 |    5.82 |     726.76 |   0.77 |            3.30 |  11.89 |  1,701,929 |    291,732 |    206 |     35.31 |      170.69 | NEGMIR     | ever_simple                | ever         |         5,060 | simple         |        25,408 |
| **NEGany~immediately_available**      | 21,078 |    0.67 |   0.72 |    5.70 |  98,046.67 |   1.22 |            1.75 | 136.37 | 72,839,589 |  3,173,660 | 29,351 |  1,278.84 |   19,799.16 | NEGATED    | immediately_available      | immediately  |        96,973 | available      |       666,909 |
| **NEGmir~ever_good**                  |    299 |    0.82 |   0.99 |    5.68 |   1,013.87 |   0.76 |            2.51 |  14.29 |  1,701,929 |    291,732 |    303 |     51.94 |      247.06 | NEGMIR     | ever_good                  | ever         |         5,060 | good           |        31,585 |
| **NEGany~any_younger**                |    255 |    0.79 |   0.83 |    5.57 |   1,323.36 |   1.28 |            2.03 |  15.13 | 72,839,589 |  3,173,660 |    307 |     13.38 |      241.62 | NEGATED    | any_younger                | any          |        34,382 | younger        |        26,216 |
| **NEGmir~ever_perfect**               |    206 |    0.82 |   1.00 |    5.57 |     714.47 |   0.76 |            2.82 |  11.88 |  1,701,929 |    291,732 |    207 |     35.48 |      170.52 | NEGMIR     | ever_perfect               | ever         |         5,060 | perfect        |         3,134 |
| **NEGany~immediately_reachable**      |    109 |    0.86 |   0.91 |    5.56 |     610.53 |   1.32 |            2.32 |   9.94 | 72,839,589 |  3,173,660 |    120 |      5.23 |      103.77 | NEGATED    | immediately_reachable      | immediately  |        96,973 | reachable      |         2,672 |
| **NEGany~ever_simple**                |    211 |    0.80 |   0.84 |    5.56 |   1,109.28 |   1.29 |            2.07 |  13.78 | 72,839,589 |  3,173,660 |    250 |     10.89 |      200.11 | NEGATED    | ever_simple                | ever         |       114,075 | simple         |       396,749 |
| **NEGany~particularly_likable**       |    119 |    0.85 |   0.89 |    5.53 |     657.49 |   1.31 |            2.26 |  10.38 | 72,839,589 |  3,173,660 |    133 |      5.79 |      113.21 | NEGATED    | particularly_likable       | particularly |       513,668 | likable        |         8,160 |
| **NEGany~ever_easy**                  |    429 |    0.73 |   0.77 |    5.41 |   2,105.13 |   1.25 |            1.87 |  19.54 | 72,839,589 |  3,173,660 |    555 |     24.18 |      404.82 | NEGATED    | ever_easy                  | ever         |       114,075 | easy           |       579,827 |
| **NEGany~immediately_sure**           |    138 |    0.82 |   0.86 |    5.40 |     738.65 |   1.30 |            2.13 |  11.15 | 72,839,589 |  3,173,660 |    160 |      6.97 |      131.03 | NEGATED    | immediately_sure           | immediately  |        96,973 | sure           |       262,825 |
| **NEGany~terribly_productive**        |     64 |    0.91 |   0.96 |    5.39 |     376.84 |   1.34 |            2.61 |   7.64 | 72,839,589 |  3,173,660 |     67 |      2.92 |       61.08 | NEGATED    | terribly_productive        | terribly     |        58,964 | productive     |       102,361 |
| **NEGany~inherently_wrong**           |  1,639 |    0.66 |   0.71 |    5.36 |   7,535.97 |   1.21 |            1.73 |  37.99 | 72,839,589 |  3,173,660 |  2,315 |    100.87 |    1,538.13 | NEGATED    | inherently_wrong           | inherently   |        47,803 | wrong          |       149,064 |
| **NEGany~particularly_likeable**      |    106 |    0.84 |   0.88 |    5.34 |     579.07 |   1.31 |            2.21 |   9.79 | 72,839,589 |  3,173,660 |    120 |      5.23 |      100.77 | NEGATED    | particularly_likeable      | particularly |       513,668 | likeable       |         5,902 |
| **NEGmir~exactly_sure**               |    148 |    0.83 |   1.00 |    5.31 |     522.11 |   0.77 |            3.16 |  10.08 |  1,701,929 |    291,732 |    148 |     25.37 |      122.63 | NEGMIR     | exactly_sure               | exactly      |         1,041 | sure           |         6,761 |
| **NEGmir~ever_enough**                |    147 |    0.83 |   1.00 |    5.30 |     518.58 |   0.77 |            3.15 |  10.05 |  1,701,929 |    291,732 |    147 |     25.20 |      121.80 | NEGMIR     | ever_enough                | ever         |         5,060 | enough         |         2,596 |
| **NEGmir~ever_certain**               |    143 |    0.83 |   1.00 |    5.26 |     504.47 |   0.77 |            3.14 |   9.91 |  1,701,929 |    291,732 |    143 |     24.51 |      118.49 | NEGMIR     | ever_certain               | ever         |         5,060 | certain        |         1,800 |
| **NEGany~terribly_sure**              |     69 |    0.86 |   0.91 |    5.08 |     386.31 |   1.32 |            2.31 |   7.91 | 72,839,589 |  3,173,660 |     76 |      3.31 |       65.69 | NEGATED    | terribly_sure              | terribly     |        58,964 | sure           |       262,825 |
| **NEGany~any_easier**                 |  1,594 |    0.62 |   0.66 |    5.08 |   6,987.78 |   1.18 |            1.64 |  37.30 | 72,839,589 |  3,173,660 |  2,405 |    104.79 |    1,489.21 | NEGATED    | any_easier                 | any          |        34,382 | easier         |       209,940 |
| **NEGany~remotely_surprising**        |     75 |    0.85 |   0.89 |    5.07 |     413.61 |   1.31 |            2.24 |   8.24 | 72,839,589 |  3,173,660 |     84 |      3.66 |       71.34 | NEGATED    | remotely_surprising        | remotely     |        16,426 | surprising     |        70,540 |
| **NEGmir~particularly_surprising**    |    166 |    0.82 |   0.99 |    5.06 |     564.67 |   0.76 |            2.51 |  10.65 |  1,701,929 |    291,732 |    168 |     28.80 |      137.20 | NEGMIR     | particularly_surprising    | particularly |        13,003 | surprising     |         2,662 |
| **NEGmir~particularly_wrong**         |    212 |    0.81 |   0.98 |    5.05 |     702.22 |   0.76 |            2.27 |  12.01 |  1,701,929 |    291,732 |    217 |     37.20 |      174.80 | NEGMIR     | particularly_wrong         | particularly |        13,003 | wrong          |        20,880 |
| **NEGmir~necessarily_wrong**          |    211 |    0.81 |   0.98 |    5.04 |     698.74 |   0.76 |            2.27 |  11.98 |  1,701,929 |    291,732 |    216 |     37.03 |      173.97 | NEGMIR     | necessarily_wrong          | necessarily  |         1,107 | wrong          |        20,880 |
| **NEGany~any_simpler**                |    226 |    0.71 |   0.75 |    5.00 |   1,087.71 |   1.24 |            1.82 |  14.16 | 72,839,589 |  3,173,660 |    300 |     13.07 |      212.93 | NEGATED    | any_simpler                | any          |        34,382 | simpler        |        23,480 |
| **NEGany~terribly_reliable**          |     51 |    0.90 |   0.94 |    4.99 |     296.70 |   1.34 |            2.51 |   6.81 | 72,839,589 |  3,173,660 |     54 |      2.35 |       48.65 | NEGATED    | terribly_reliable          | terribly     |        58,964 | reliable       |        90,598 |
| **NEGany~particularly_forthcoming**   |     72 |    0.82 |   0.87 |    4.85 |     387.25 |   1.30 |            2.14 |   8.06 | 72,839,589 |  3,173,660 |     83 |      3.62 |       68.38 | NEGATED    | particularly_forthcoming   | particularly |       513,668 | forthcoming    |         7,473 |
| **NEGany~ever_certain**               |    147 |    0.72 |   0.76 |    4.80 |     713.34 |   1.24 |            1.84 |  11.43 | 72,839,589 |  3,173,660 |    193 |      8.41 |      138.59 | NEGATED    | ever_certain               | ever         |       114,075 | certain        |        74,952 |
| **NEGany~particularly_athletic**      |    108 |    0.75 |   0.79 |    4.77 |     541.01 |   1.26 |            1.92 |   9.82 | 72,839,589 |  3,173,660 |    136 |      5.93 |      102.07 | NEGATED    | particularly_athletic      | particularly |       513,668 | athletic       |        17,142 |
| **NEGany~any_nicer**                  |     96 |    0.76 |   0.81 |    4.75 |     486.82 |   1.27 |            1.95 |   9.27 | 72,839,589 |  3,173,660 |    119 |      5.18 |       90.82 | NEGATED    | any_nicer                  | any          |        34,382 | nicer          |         9,955 |
| **NEGany~particularly_surprising**    |  1,069 |    0.57 |   0.61 |    4.71 |   4,433.52 |   1.15 |            1.54 |  30.37 | 72,839,589 |  3,173,660 |  1,743 |     75.94 |      993.06 | NEGATED    | particularly_surprising    | particularly |       513,668 | surprising     |        70,540 |
| **NEGany~immediately_able**           |    626 |    0.59 |   0.63 |    4.70 |   2,655.18 |   1.16 |            1.58 |  23.30 | 72,839,589 |  3,173,660 |    989 |     43.09 |      582.91 | NEGATED    | immediately_able           | immediately  |        96,973 | able           |       223,196 |
| **NEGany~inherently_bad**             |    794 |    0.56 |   0.61 |    4.62 |   3,270.68 |   1.14 |            1.53 |  26.16 | 72,839,589 |  3,173,660 |  1,307 |     56.95 |      737.05 | NEGATED    | inherently_bad             | inherently   |        47,803 | bad            |       429,537 |
| **NEGany~remotely_true**              |    250 |    0.63 |   0.68 |    4.61 |   1,111.13 |   1.19 |            1.66 |  14.79 | 72,839,589 |  3,173,660 |    370 |     16.12 |      233.88 | NEGATED    | remotely_true              | remotely     |        16,426 | true           |       231,639 |
| **NEGany~immediately_obvious**        |  2,238 |    0.52 |   0.56 |    4.55 |   8,712.07 |   1.11 |            1.45 |  43.63 | 72,839,589 |  3,173,660 |  3,989 |    173.80 |    2,064.20 | NEGATED    | immediately_obvious        | immediately  |        96,973 | obvious        |       165,439 |
| **NEGany~particularly_revolutionary** |     77 |    0.75 |   0.79 |    4.49 |     385.60 |   1.26 |            1.92 |   8.29 | 72,839,589 |  3,173,660 |     97 |      4.23 |       72.77 | NEGATED    | particularly_revolutionary | particularly |       513,668 | revolutionary  |        10,338 |
| **NEGany~ever_boring**                |     72 |    0.76 |   0.80 |    4.46 |     362.74 |   1.26 |            1.93 |   8.02 | 72,839,589 |  3,173,660 |     90 |      3.92 |       68.08 | NEGATED    | ever_boring                | ever         |       114,075 | boring         |        45,891 |
| **NEGmir~any_better**                 |    380 |    0.74 |   0.91 |    4.38 |   1,096.01 |   0.72 |            1.67 |  15.81 |  1,701,929 |    291,732 |    419 |     71.82 |      308.18 | NEGMIR     | any_better                 | any          |         1,197 | better         |        14,013 |
| **NEGmir~that_simple**                |    474 |    0.72 |   0.90 |    4.36 |   1,340.19 |   0.72 |            1.62 |  17.61 |  1,701,929 |    291,732 |    529 |     90.68 |      383.32 | NEGMIR     | that_simple                | that         |         5,494 | simple         |        25,408 |
| **NEGany~any_clearer**                |    355 |    0.54 |   0.58 |    4.26 |   1,421.60 |   1.13 |            1.49 |  17.44 | 72,839,589 |  3,173,660 |    608 |     26.49 |      328.51 | NEGATED    | any_clearer                | any          |        34,382 | clearer        |        11,680 |
| **NEGmir~particularly_remarkable**    |    108 |    0.80 |   0.97 |    4.24 |     354.53 |   0.75 |            2.18 |   8.56 |  1,701,929 |    291,732 |    111 |     19.03 |       88.97 | NEGMIR     | particularly_remarkable    | particularly |        13,003 | remarkable     |         3,238 |
| **NEGmir~that_easy**                  |    450 |    0.71 |   0.89 |    4.23 |   1,248.84 |   0.71 |            1.57 |  17.11 |  1,701,929 |    291,732 |    508 |     87.08 |      362.92 | NEGMIR     | that_easy                  | that         |         5,494 | easy           |        18,610 |
| **NEGany~immediately_certain**        |     70 |    0.71 |   0.75 |    4.19 |     336.68 |   1.24 |            1.82 |   7.88 | 72,839,589 |  3,173,660 |     93 |      4.05 |       65.95 | NEGATED    | immediately_certain        | immediately  |        96,973 | certain        |        74,952 |
| **NEGmir~ever_able**                  |    136 |    0.78 |   0.95 |    4.17 |     426.52 |   0.74 |            1.94 |   9.56 |  1,701,929 |    291,732 |    143 |     24.51 |      111.49 | NEGMIR     | ever_able                  | ever         |         5,060 | able           |         3,704 |
| **NEGany~remotely_ready**             |     58 |    0.74 |   0.78 |    4.16 |     287.63 |   1.26 |            1.89 |   7.19 | 72,839,589 |  3,173,660 |     74 |      3.22 |       54.78 | NEGATED    | remotely_ready             | remotely     |        16,426 | ready          |       141,590 |
| **NEGany~remotely_funny**             |    137 |    0.60 |   0.65 |    4.15 |     589.74 |   1.17 |            1.60 |  10.92 | 72,839,589 |  3,173,660 |    212 |      9.24 |      127.76 | NEGATED    | remotely_funny             | remotely     |        16,426 | funny          |       122,927 |
| **NEGmir~inherently_wrong**           |  1,513 |    0.66 |   0.83 |    4.08 |   3,787.53 |   0.68 |            1.37 |  30.85 |  1,701,929 |    291,732 |  1,826 |    313.00 |    1,200.00 | NEGMIR     | inherently_wrong           | inherently   |         5,133 | wrong          |        20,880 |
| **NEGany~particularly_flashy**        |     57 |    0.73 |   0.77 |    4.08 |     278.96 |   1.25 |            1.86 |   7.12 | 72,839,589 |  3,173,660 |     74 |      3.22 |       53.78 | NEGATED    | particularly_flashy        | particularly |       513,668 | flashy         |         4,494 |
| **NEGmir~terribly_surprising**        |     67 |    0.83 |   1.00 |    4.07 |     236.35 |   0.77 |            2.81 |   6.78 |  1,701,929 |    291,732 |     67 |     11.48 |       55.52 | NEGMIR     | terribly_surprising        | terribly     |         4,610 | surprising     |         2,662 |
| **NEGany~any_cheaper**                |    129 |    0.58 |   0.63 |    4.01 |     542.97 |   1.16 |            1.56 |  10.57 | 72,839,589 |  3,173,660 |    206 |      8.98 |      120.02 | NEGATED    | any_cheaper                | any          |        34,382 | cheaper        |        46,055 |
| **NEGmir~particularly_close**         |    136 |    0.77 |   0.94 |    4.01 |     415.70 |   0.74 |            1.84 |   9.53 |  1,701,929 |    291,732 |    145 |     24.85 |      111.15 | NEGMIR     | particularly_close         | particularly |        13,003 | close          |        13,874 |
| **NEGany~any_smarter**                |     89 |    0.63 |   0.67 |    4.00 |     394.95 |   1.19 |            1.65 |   8.82 | 72,839,589 |  3,173,660 |    132 |      5.75 |       83.25 | NEGATED    | any_smarter                | any          |        34,382 | smarter        |         8,501 |
| **NEGany~any_worse**                  |  1,686 |    0.42 |   0.46 |    3.94 |   5,676.37 |   1.02 |            1.27 |  37.16 | 72,839,589 |  3,173,660 |  3,673 |    160.03 |    1,525.97 | NEGATED    | any_worse                  | any          |        34,382 | worse          |       179,012 |
| **NEGany~any_brighter**               |     63 |    0.67 |   0.72 |    3.91 |     292.00 |   1.22 |            1.74 |   7.45 | 72,839,589 |  3,173,660 |     88 |      3.83 |       59.17 | NEGATED    | any_brighter               | any          |        34,382 | brighter       |         9,280 |
| **NEGany~immediately_forthcoming**    |    133 |    0.55 |   0.60 |    3.89 |     540.70 |   1.14 |            1.51 |  10.69 | 72,839,589 |  3,173,660 |    223 |      9.72 |      123.28 | NEGATED    | immediately_forthcoming    | immediately  |        96,973 | forthcoming    |         7,473 |
| **NEGany~ever_good**                  |    331 |    0.46 |   0.50 |    3.81 |   1,188.69 |   1.06 |            1.34 |  16.61 | 72,839,589 |  3,173,660 |    660 |     28.76 |      302.24 | NEGATED    | ever_good                  | ever         |       114,075 | good           |     1,681,795 |
| **NEGmir~ever_boring**                |     57 |    0.83 |   1.00 |    3.80 |     201.07 |   0.77 |            2.75 |   6.26 |  1,701,929 |    291,732 |     57 |      9.77 |       47.23 | NEGMIR     | ever_boring                | ever         |         5,060 | boring         |         1,961 |
| **NEGmir~ever_black**                 |     56 |    0.83 |   1.00 |    3.77 |     197.54 |   0.77 |            2.74 |   6.20 |  1,701,929 |    291,732 |     56 |      9.60 |       46.40 | NEGMIR     | ever_black                 | ever         |         5,060 | black          |         1,412 |
| **NEGmir~particularly_novel**         |     54 |    0.83 |   1.00 |    3.71 |     190.49 |   0.77 |            2.72 |   6.09 |  1,701,929 |    291,732 |     54 |      9.26 |       44.74 | NEGMIR     | particularly_novel         | particularly |        13,003 | novel          |           320 |
| **NEGmir~terribly_new**               |     69 |    0.80 |   0.97 |    3.66 |     225.93 |   0.75 |            2.13 |   6.84 |  1,701,929 |    291,732 |     71 |     12.17 |       56.83 | NEGMIR     | terribly_new               | terribly     |         4,610 | new            |        12,836 |
| **NEGany~remotely_close**             |    694 |    0.39 |   0.43 |    3.65 |   2,243.22 |   1.00 |            1.23 |  23.70 | 72,839,589 |  3,173,660 |  1,597 |     69.58 |      624.42 | NEGATED    | remotely_close             | remotely     |        16,426 | close          |       411,329 |
| **NEGmir~that_great**                 |    286 |    0.66 |   0.84 |    3.57 |     725.16 |   0.69 |            1.39 |  13.45 |  1,701,929 |    291,732 |    342 |     58.62 |      227.38 | NEGMIR     | that_great                 | that         |         5,494 | great          |         5,568 |
| **NEGany~ever_enough**                |    173 |    0.45 |   0.50 |    3.54 |     618.62 |   1.06 |            1.34 |  12.00 | 72,839,589 |  3,173,660 |    347 |     15.12 |      157.88 | NEGATED    | ever_enough                | ever         |       114,075 | enough         |       152,020 |
| **COM~ever_closer**                   |  6,305 |    0.04 |   1.00 |    3.51 |     538.66 |   0.02 |            2.06 |   3.44 | 72,839,589 | 69,662,736 |  6,307 |  6,031.92 |      273.08 | COMPLEMENT | ever_closer                | ever         |       114,075 | closer         |        61,475 |
| **NEGany~inherently_illegal**         |     59 |    0.60 |   0.64 |    3.51 |     252.59 |   1.17 |            1.59 |   7.16 | 72,839,589 |  3,173,660 |     92 |      4.01 |       54.99 | NEGATED    | inherently_illegal         | inherently   |        47,803 | illegal        |        30,194 |
| **NEGmir~terribly_original**          |     45 |    0.83 |   1.00 |    3.40 |     158.74 |   0.77 |            2.64 |   5.56 |  1,701,929 |    291,732 |     45 |      7.71 |       37.29 | NEGMIR     | terribly_original          | terribly     |         4,610 | original       |         1,555 |
| **NEGmir~exactly_clear**              |     52 |    0.81 |   0.98 |    3.38 |     173.89 |   0.76 |            2.23 |   5.95 |  1,701,929 |    291,732 |     53 |      9.08 |       42.92 | NEGMIR     | exactly_clear              | exactly      |         1,041 | clear          |         6,722 |
| **NEGmir~particularly_comfortable**   |     44 |    0.83 |   1.00 |    3.36 |     155.21 |   0.77 |            2.63 |   5.50 |  1,701,929 |    291,732 |     44 |      7.54 |       36.46 | NEGMIR     | particularly_comfortable   | particularly |        13,003 | comfortable    |         4,642 |
| **NEGany~ever_perfect**               |    216 |    0.40 |   0.44 |    3.34 |     706.71 |   1.01 |            1.24 |  13.25 | 72,839,589 |  3,173,660 |    489 |     21.31 |      194.69 | NEGATED    | ever_perfect               | ever         |       114,075 | perfect        |       104,659 |
| **NEGany~ever_sure**                  |     87 |    0.49 |   0.54 |    3.34 |     328.20 |   1.09 |            1.41 |   8.57 | 72,839,589 |  3,173,660 |    162 |      7.06 |       79.94 | NEGATED    | ever_sure                  | ever         |       114,075 | sure           |       262,825 |
| **NEGmir~remotely_close**             |    218 |    0.65 |   0.82 |    3.28 |     532.96 |   0.68 |            1.33 |  11.67 |  1,701,929 |    291,732 |    267 |     45.77 |      172.23 | NEGMIR     | remotely_close             | remotely     |         2,341 | close          |        13,874 |
| **NEGmir~any_different**              |     48 |    0.81 |   0.98 |    3.24 |     159.93 |   0.76 |            2.19 |   5.72 |  1,701,929 |    291,732 |     49 |      8.40 |       39.60 | NEGMIR     | any_different              | any          |         1,197 | different      |        36,166 |
| **NEGany~immediately_intuitive**      |     54 |    0.55 |   0.59 |    3.21 |     218.74 |   1.13 |            1.50 |   6.81 | 72,839,589 |  3,173,660 |     91 |      3.96 |       50.04 | NEGATED    | immediately_intuitive      | immediately  |        96,973 | intuitive      |        20,664 |
| **NEGmir~that_big**                   |    113 |    0.69 |   0.86 |    3.17 |     300.54 |   0.70 |            1.47 |   8.52 |  1,701,929 |    291,732 |    131 |     22.46 |       90.54 | NEGMIR     | that_big                   | that         |         5,494 | big            |         8,177 |
| **NEGmir~that_popular**               |     65 |    0.76 |   0.93 |    3.15 |     195.15 |   0.73 |            1.76 |   6.57 |  1,701,929 |    291,732 |     70 |     12.00 |       53.00 | NEGMIR     | that_popular               | that         |         5,494 | popular        |         5,668 |
| **NEGmir~ever_sick**                  |     39 |    0.83 |   1.00 |    3.15 |     137.57 |   0.77 |            2.58 |   5.17 |  1,701,929 |    291,732 |     39 |      6.69 |       32.31 | NEGMIR     | ever_sick                  | ever         |         5,060 | sick           |         1,895 |
| **NEGmir~particularly_revolutionary** |     39 |    0.83 |   1.00 |    3.15 |     137.57 |   0.77 |            2.58 |   5.17 |  1,701,929 |    291,732 |     39 |      6.69 |       32.31 | NEGMIR     | particularly_revolutionary | particularly |        13,003 | revolutionary  |           485 |
| **NEGany~remotely_qualified**         |     57 |    0.52 |   0.56 |    3.13 |     222.79 |   1.11 |            1.45 |   6.97 | 72,839,589 |  3,173,660 |    101 |      4.40 |       52.60 | NEGATED    | remotely_qualified         | remotely     |        16,426 | qualified      |        74,643 |
| **NEGmir~remotely_true**              |     61 |    0.75 |   0.92 |    3.04 |     181.65 |   0.73 |            1.73 |   6.36 |  1,701,929 |    291,732 |     66 |     11.31 |       49.69 | NEGMIR     | remotely_true              | remotely     |         2,341 | true           |         6,191 |
| **NEGmir~any_easier**                 |     61 |    0.75 |   0.92 |    3.04 |     181.65 |   0.73 |            1.73 |   6.36 |  1,701,929 |    291,732 |     66 |     11.31 |       49.69 | NEGMIR     | any_easier                 | any          |         1,197 | easier         |         2,386 |
| **NEGany~remotely_comparable**        |    118 |    0.38 |   0.43 |    2.98 |     375.73 |   0.99 |            1.21 |   9.75 | 72,839,589 |  3,173,660 |    277 |     12.07 |      105.93 | NEGATED    | remotely_comparable        | remotely     |        16,426 | comparable     |        12,252 |
| **NEGmir~necessarily_bad**            |     50 |    0.77 |   0.94 |    2.95 |     154.45 |   0.74 |            1.84 |   5.79 |  1,701,929 |    291,732 |     53 |      9.08 |       40.92 | NEGMIR     | necessarily_bad            | necessarily  |         1,107 | bad            |        10,261 |
| **NEGmir~inherently_bad**             |    148 |    0.62 |   0.79 |    2.86 |     342.53 |   0.66 |            1.25 |   9.52 |  1,701,929 |    291,732 |    188 |     32.23 |      115.77 | NEGMIR     | inherently_bad             | inherently   |         5,133 | bad            |        10,261 |
| **NEGany~inherently_evil**            |    358 |    0.28 |   0.32 |    2.81 |     905.80 |   0.86 |            1.01 |  16.33 | 72,839,589 |  3,173,660 |  1,123 |     48.93 |      309.07 | NEGATED    | inherently_evil            | inherently   |        47,803 | evil           |        22,706 |
| **NEGany~remotely_interested**        |    330 |    0.27 |   0.31 |    2.74 |     817.06 |   0.85 |            1.00 |  15.62 | 72,839,589 |  3,173,660 |  1,062 |     46.27 |      283.73 | NEGATED    | remotely_interested        | remotely     |        16,426 | interested     |       264,528 |
| **NEGmir~any_worse**                  |     87 |    0.66 |   0.83 |    2.73 |     217.46 |   0.68 |            1.36 |   7.40 |  1,701,929 |    291,732 |    105 |     18.00 |       69.00 | NEGMIR     | any_worse                  | any          |         1,197 | worse          |         8,790 |
| **NEGmir~particularly_radical**       |     31 |    0.83 |   1.00 |    2.73 |     109.35 |   0.77 |            2.48 |   4.61 |  1,701,929 |    291,732 |     31 |      5.31 |       25.69 | NEGMIR     | particularly_radical       | particularly |        13,003 | radical        |         1,072 |
| **NEGmir~that_keen**                  |     31 |    0.83 |   1.00 |    2.73 |     109.35 |   0.77 |            2.48 |   4.61 |  1,701,929 |    291,732 |     31 |      5.31 |       25.69 | NEGMIR     | that_keen                  | that         |         5,494 | keen           |         1,360 |
| **NEGmir~any_bigger**                 |     36 |    0.80 |   0.97 |    2.73 |     118.17 |   0.75 |            2.07 |   4.94 |  1,701,929 |    291,732 |     37 |      6.34 |       29.66 | NEGMIR     | any_bigger                 | any          |         1,197 | bigger         |         3,923 |
| **NEGmir~remotely_comparable**        |     44 |    0.76 |   0.94 |    2.72 |     134.02 |   0.74 |            1.79 |   5.42 |  1,701,929 |    291,732 |     47 |      8.06 |       35.94 | NEGMIR     | remotely_comparable        | remotely     |         2,341 | comparable     |           283 |
| **NEGmir~necessarily_true**           |     53 |    0.73 |   0.90 |    2.69 |     150.42 |   0.72 |            1.60 |   5.89 |  1,701,929 |    291,732 |     59 |     10.11 |       42.89 | NEGMIR     | necessarily_true           | necessarily  |         1,107 | true           |         6,191 |
| **NEGany~ever_satisfied**             |     64 |    0.38 |   0.42 |    2.57 |     203.01 |   0.99 |            1.21 |   7.18 | 72,839,589 |  3,173,660 |    151 |      6.58 |       57.42 | NEGATED    | ever_satisfied             | ever         |       114,075 | satisfied      |        62,862 |
| **NEGmir~yet_available**              |     28 |    0.83 |   1.00 |    2.54 |      98.77 |   0.77 |            2.44 |   4.38 |  1,701,929 |    291,732 |     28 |      4.80 |       23.20 | NEGMIR     | yet_available              | yet          |           815 | available      |        10,284 |
| **NEGany~inherently_negative**        |     75 |    0.35 |   0.39 |    2.50 |     222.63 |   0.95 |            1.15 |   7.69 | 72,839,589 |  3,173,660 |    193 |      8.41 |       66.59 | NEGATED    | inherently_negative        | inherently   |        47,803 | negative       |        53,385 |
| **NEGmir~terribly_interesting**       |     56 |    0.68 |   0.85 |    2.44 |     145.16 |   0.69 |            1.42 |   5.97 |  1,701,929 |    291,732 |     66 |     11.31 |       44.69 | NEGMIR     | terribly_interesting       | terribly     |         4,610 | interesting    |        12,447 |
| **NEGmir~particularly_fast**          |     26 |    0.83 |   1.00 |    2.39 |      91.71 |   0.77 |            2.41 |   4.22 |  1,701,929 |    291,732 |     26 |      4.46 |       21.54 | NEGMIR     | particularly_fast          | particularly |        13,003 | fast           |         1,259 |
| **NEGmir~terribly_interested**        |     39 |    0.74 |   0.91 |    2.36 |     112.46 |   0.72 |            1.63 |   5.06 |  1,701,929 |    291,732 |     43 |      7.37 |       31.63 | NEGMIR     | terribly_interested        | terribly     |         4,610 | interested     |         8,255 |
| **NEGmir~any_closer**                 |     57 |    0.65 |   0.83 |    2.33 |     141.82 |   0.68 |            1.35 |   5.98 |  1,701,929 |    291,732 |     69 |     11.83 |       45.17 | NEGMIR     | any_closer                 | any          |         1,197 | closer         |           993 |
| **NEGmir~exactly_new**                |     29 |    0.80 |   0.97 |    2.31 |      93.90 |   0.75 |            1.98 |   4.43 |  1,701,929 |    291,732 |     30 |      5.14 |       23.86 | NEGMIR     | exactly_new                | exactly      |         1,041 | new            |        12,836 |
| **NEGmir~terribly_unusual**           |     24 |    0.83 |   1.00 |    2.23 |      84.66 |   0.77 |            2.37 |   4.06 |  1,701,929 |    291,732 |     24 |      4.11 |       19.89 | NEGMIR     | terribly_unusual           | terribly     |         4,610 | unusual        |         2,302 |
| **NEGmir~terribly_special**           |     24 |    0.83 |   1.00 |    2.23 |      84.66 |   0.77 |            2.37 |   4.06 |  1,701,929 |    291,732 |     24 |      4.11 |       19.89 | NEGMIR     | terribly_special           | terribly     |         4,610 | special        |        15,541 |
| **NEGany~inherently_good**            |    283 |    0.20 |   0.24 |    2.21 |     554.72 |   0.74 |            0.84 |  13.77 | 72,839,589 |  3,173,660 |  1,177 |     51.28 |      231.72 | NEGATED    | inherently_good            | inherently   |        47,803 | good           |     1,681,795 |
| **NEGany~remotely_similar**           |    152 |    0.23 |   0.27 |    2.20 |     334.61 |   0.80 |            0.91 |  10.35 | 72,839,589 |  3,173,660 |    559 |     24.36 |      127.64 | NEGATED    | remotely_similar           | remotely     |        16,426 | similar        |       203,453 |
| **NEGmir~necessarily_right**          |     23 |    0.83 |   1.00 |    2.15 |      81.13 |   0.77 |            2.36 |   3.97 |  1,701,929 |    291,732 |     23 |      3.94 |       19.06 | NEGMIR     | necessarily_right          | necessarily  |         1,107 | right          |         5,576 |
| **NEGmir~that_impressive**            |     23 |    0.83 |   1.00 |    2.15 |      81.13 |   0.77 |            2.36 |   3.97 |  1,701,929 |    291,732 |     23 |      3.94 |       19.06 | NEGMIR     | that_impressive            | that         |         5,494 | impressive     |         5,007 |
| **NEGmir~any_good**                   |     32 |    0.74 |   0.91 |    2.12 |      93.53 |   0.73 |            1.65 |   4.60 |  1,701,929 |    291,732 |     35 |      6.00 |       26.00 | NEGMIR     | any_good                   | any          |         1,197 | good           |        31,585 |
| **NEGany~remotely_accurate**          |     50 |    0.33 |   0.37 |    2.10 |     143.78 |   0.93 |            1.12 |   6.25 | 72,839,589 |  3,173,660 |    134 |      5.84 |       44.16 | NEGATED    | remotely_accurate          | remotely     |        16,426 | accurate       |       152,299 |
| **NEGmir~inherently_illegal**         |     26 |    0.79 |   0.96 |    2.10 |      83.54 |   0.75 |            1.93 |   4.19 |  1,701,929 |    291,732 |     27 |      4.63 |       21.37 | NEGMIR     | inherently_illegal         | inherently   |         5,133 | illegal        |           937 |
| **NEGmir~remotely_believable**        |     21 |    0.83 |   1.00 |    1.96 |      74.08 |   0.77 |            2.32 |   3.80 |  1,701,929 |    291,732 |     21 |      3.60 |       17.40 | NEGMIR     | remotely_believable        | remotely     |         2,341 | believable     |           600 |
| **NEGmir~yet_certain**                |     21 |    0.83 |   1.00 |    1.96 |      74.08 |   0.77 |            2.32 |   3.80 |  1,701,929 |    291,732 |     21 |      3.60 |       17.40 | NEGMIR     | yet_certain                | yet          |           815 | certain        |         1,800 |
| **NEGmir~any_younger**                |     20 |    0.83 |   1.00 |    1.86 |      70.55 |   0.77 |            2.30 |   3.71 |  1,701,929 |    291,732 |     20 |      3.43 |       16.57 | NEGMIR     | any_younger                | any          |         1,197 | younger        |           939 |
| **NEGmir~exactly_easy**               |     20 |    0.83 |   1.00 |    1.86 |      70.55 |   0.77 |            2.30 |   3.71 |  1,701,929 |    291,732 |     20 |      3.43 |       16.57 | NEGMIR     | exactly_easy               | exactly      |         1,041 | easy           |        18,610 |
| **NEGmir~remotely_funny**             |     41 |    0.63 |   0.80 |    1.85 |      97.91 |   0.67 |            1.28 |   5.04 |  1,701,929 |    291,732 |     51 |      8.74 |       32.26 | NEGMIR     | remotely_funny             | remotely     |         2,341 | funny          |         5,365 |
| **NEGmir~immediately_available**      |    162 |    0.42 |   0.59 |    1.85 |     241.53 |   0.54 |            0.84 |   9.02 |  1,701,929 |    291,732 |    275 |     47.14 |      114.86 | NEGMIR     | immediately_available      | immediately  |         1,195 | available      |        10,284 |
| **NEGmir~that_fond**                  |     23 |    0.79 |   0.96 |    1.84 |      73.19 |   0.75 |            1.88 |   3.94 |  1,701,929 |    291,732 |     24 |      4.11 |       18.89 | NEGMIR     | that_fond                  | that         |         5,494 | fond           |         1,115 |
| **NEGmir~that_comfortable**           |     23 |    0.79 |   0.96 |    1.84 |      73.19 |   0.75 |            1.88 |   3.94 |  1,701,929 |    291,732 |     24 |      4.11 |       18.89 | NEGMIR     | that_comfortable           | that         |         5,494 | comfortable    |         4,642 |
| **NEGmir~necessarily_new**            |     23 |    0.79 |   0.96 |    1.84 |      73.19 |   0.75 |            1.88 |   3.94 |  1,701,929 |    291,732 |     24 |      4.11 |       18.89 | NEGMIR     | necessarily_new            | necessarily  |         1,107 | new            |        12,836 |
| **NEGmir~remotely_interesting**       |     56 |    0.56 |   0.73 |    1.80 |     115.20 |   0.63 |            1.10 |   5.72 |  1,701,929 |    291,732 |     77 |     13.20 |       42.80 | NEGMIR     | remotely_interesting       | remotely     |         2,341 | interesting    |        12,447 |
| **NEGmir~terribly_popular**           |     19 |    0.83 |   1.00 |    1.75 |      67.02 |   0.77 |            2.28 |   3.61 |  1,701,929 |    291,732 |     19 |      3.26 |       15.74 | NEGMIR     | terribly_popular           | terribly     |         4,610 | popular        |         5,668 |
| **NEGmir~yet_clear**                  |     19 |    0.83 |   1.00 |    1.75 |      67.02 |   0.77 |            2.28 |   3.61 |  1,701,929 |    291,732 |     19 |      3.26 |       15.74 | NEGMIR     | yet_clear                  | yet          |           815 | clear          |         6,722 |
| **NEGmir~yet_sure**                   |     19 |    0.83 |   1.00 |    1.75 |      67.02 |   0.77 |            2.28 |   3.61 |  1,701,929 |    291,732 |     19 |      3.26 |       15.74 | NEGMIR     | yet_sure                   | yet          |           815 | sure           |         6,761 |
| **NEGmir~remotely_similar**           |     71 |    0.49 |   0.66 |    1.70 |     127.32 |   0.59 |            0.98 |   6.25 |  1,701,929 |    291,732 |    107 |     18.34 |       52.66 | NEGMIR     | remotely_similar           | remotely     |         2,341 | similar        |         7,011 |
| **NEGmir~inherently_better**          |     44 |    0.57 |   0.75 |    1.65 |      93.95 |   0.64 |            1.14 |   5.11 |  1,701,929 |    291,732 |     59 |     10.11 |       33.89 | NEGMIR     | inherently_better          | inherently   |         5,133 | better         |        14,013 |
| **NEGmir~necessarily_better**         |     27 |    0.70 |   0.87 |    1.63 |      72.90 |   0.71 |            1.47 |   4.17 |  1,701,929 |    291,732 |     31 |      5.31 |       21.69 | NEGMIR     | necessarily_better         | necessarily  |         1,107 | better         |        14,013 |
| **NEGmir~inherently_improper**        |     18 |    0.83 |   1.00 |    1.63 |      63.49 |   0.77 |            2.25 |   3.52 |  1,701,929 |    291,732 |     18 |      3.09 |       14.91 | NEGMIR     | inherently_improper        | inherently   |         5,133 | improper       |           142 |
| **NEGmir~yet_ready**                  |     18 |    0.83 |   1.00 |    1.63 |      63.49 |   0.77 |            2.25 |   3.52 |  1,701,929 |    291,732 |     18 |      3.09 |       14.91 | NEGMIR     | yet_ready                  | yet          |           815 | ready          |         3,034 |
| **NEGmir~any_clearer**                |     17 |    0.83 |   1.00 |    1.50 |      59.97 |   0.77 |            2.23 |   3.42 |  1,701,929 |    291,732 |     17 |      2.91 |       14.09 | NEGMIR     | any_clearer                | any          |         1,197 | clearer        |           130 |
| **NEGmir~remotely_surprising**        |     17 |    0.83 |   1.00 |    1.50 |      59.97 |   0.77 |            2.23 |   3.42 |  1,701,929 |    291,732 |     17 |      2.91 |       14.09 | NEGMIR     | remotely_surprising        | remotely     |         2,341 | surprising     |         2,662 |
| **NEGmir~any_higher**                 |     21 |    0.74 |   0.91 |    1.42 |      61.24 |   0.73 |            1.62 |   3.72 |  1,701,929 |    291,732 |     23 |      3.94 |       17.06 | NEGMIR     | any_higher                 | any          |         1,197 | higher         |         2,893 |
| **NEGmir~remotely_possible**          |     38 |    0.56 |   0.73 |    1.42 |      78.73 |   0.63 |            1.11 |   4.72 |  1,701,929 |    291,732 |     52 |      8.91 |       29.09 | NEGMIR     | remotely_possible          | remotely     |         2,341 | possible       |         3,160 |
| **NEGmir~that_clear**                 |     18 |    0.78 |   0.95 |    1.30 |      56.03 |   0.74 |            1.78 |   3.47 |  1,701,929 |    291,732 |     19 |      3.26 |       14.74 | NEGMIR     | that_clear                 | that         |         5,494 | clear          |         6,722 |
| **NEGmir~necessarily_illegal**        |     15 |    0.83 |   1.00 |    1.20 |      52.91 |   0.77 |            2.18 |   3.21 |  1,701,929 |    291,732 |     15 |      2.57 |       12.43 | NEGMIR     | necessarily_illegal        | necessarily  |         1,107 | illegal        |           937 |
| **NEGmir~terribly_clear**             |     15 |    0.83 |   1.00 |    1.20 |      52.91 |   0.77 |            2.18 |   3.21 |  1,701,929 |    291,732 |     15 |      2.57 |       12.43 | NEGMIR     | terribly_clear             | terribly     |         4,610 | clear          |         6,722 |
| **NEGmir~inherently_evil**            |     58 |    0.41 |   0.59 |    1.18 |      85.70 |   0.53 |            0.83 |   5.39 |  1,701,929 |    291,732 |     99 |     16.97 |       41.03 | NEGMIR     | inherently_evil            | inherently   |         5,133 | evil           |         1,271 |
| **NEGany~inherently_problematic**     |     58 |    0.17 |   0.21 |    1.17 |      98.70 |   0.68 |            0.77 |   6.03 | 72,839,589 |  3,173,660 |    277 |     12.07 |       45.93 | NEGATED    | inherently_problematic     | inherently   |        47,803 | problematic    |        33,408 |
| **NEGmir~immediately_clear**          |     31 |    0.55 |   0.72 |    1.13 |      62.95 |   0.62 |            1.09 |   4.24 |  1,701,929 |    291,732 |     43 |      7.37 |       23.63 | NEGMIR     | immediately_clear          | immediately  |         1,195 | clear          |         6,722 |
| **NEGmir~terribly_remarkable**        |     14 |    0.83 |   1.00 |    1.03 |      49.38 |   0.77 |            2.15 |   3.10 |  1,701,929 |    291,732 |     14 |      2.40 |       11.60 | NEGMIR     | terribly_remarkable        | terribly     |         4,610 | remarkable     |         3,238 |
| **NEGmir~remotely_new**               |     19 |    0.69 |   0.86 |    1.00 |      50.62 |   0.70 |            1.43 |   3.49 |  1,701,929 |    291,732 |     22 |      3.77 |       15.23 | NEGMIR     | remotely_new               | remotely     |         2,341 | new            |        12,836 |



In [10]:
nb_show_table(NEG_bigrams_sample.l1.value_counts().to_frame('subtotal in selected bigrams'))


|                |   `subtotal in selected bigrams` |
|:---------------|---------------------------------:|
| **NEGATED**    |                              106 |
| **NEGMIR**     |                               84 |
| **COMPLEMENT** |                                1 |




|                |   `subtotal in selected bigrams` |
|:---------------|---------------------------------:|
| **NEGATED**    |                              106 |
| **NEGMIR**     |                               84 |
| **COMPLEMENT** |                                1 |



In [11]:
nb_show_table(NEG_bigrams_sample.filter(like='O', axis=0).T)


|                 | `COM~ever_closer`    |
|:----------------|:---------------------|
| **f**           | 6305                 |
| **dP1**         | 0.043301016092300415 |
| **P1**          | 0.9996829032897949   |
| **LRC**         | 3.5134613513946533   |
| **G2**          | 538.66082            |
| **MI**          | 0.019229218363761902 |
| **odds_r_disc** | 2.0608150959014893   |
| **t**           | 3.439067840576172    |
| **N**           | 72839589             |
| **f1**          | 69662736             |
| **f2**          | 6307                 |
| **exp_f**       | 6031.9241            |
| **unexp_f**     | 273.0758999999998    |
| **l1**          | COMPLEMENT           |
| **l2**          | ever_closer          |
| **adv**         | ever                 |
| **adv_total**   | 114075               |
| **adj**         | closer               |
| **adj_total**   | 61475                |



```python
nb_show_table(NEG_bigrams_sample.filter(like='O', axis=0).T)
```
(_+ manual sort & round_)

|                 | `COM~ever_closer` |
|:----------------|------------------:|
| **N**           |        72,839,589 |
| **f**           |             6,305 |
| **l1**          |        COMPLEMENT |
| **l2**          |       ever_closer |
| **f1**          |        69,662,736 |
| **f2**          |             6,307 |
| **adv**         |              ever |
| **adj**         |            closer |
| **adv_total**   |           114,075 |
| **adj_total**   |            61,475 |
| **exp_f**       |             6,032 |
| **unexp_f**     |               273 |
| **P1**          |            0.9997 |
| **dP1**         |            0.0433 |
| **LRC**         |              3.51 |
| **G2**          |             538.6 |
| **odds_r_disc** |              2.06 |
| **MI**          |            0.0192 |
| **t**           |              3.44 |

